In [1]:
import glob
import re
import subprocess
import os
from datetime import datetime
from graphviz import Digraph        
from datetime import datetime

# Global Variables

In [2]:
STRACE_TIME_FORMAT = '%H:%M:%S.%f' #03:05:34.993967

In [3]:
###### regex #########
RE_PID = 'log.([0-9]*)' # strace.log.(1242).exec
RE_TIME = '^(.*?) ' # (04:31:48.580504) execve(...)
RE_EXECVE_COMMAND = '\", \[\"(.*?)\"' # "/bin/sed", ["(sed)", "-i", "-e", "/exit/d", "/etc/rc.local"], ["SHLVL=1", "_=/usr/bin/strace", "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "LANG=C.UTF-8", "PWD=/"]
RE_EXECVE_OPTION = '\", \[\".*?\", (.*?)\],' # "/bin/sed", [("sed", "-i", "-e", "/exit/d", "/etc/rc.local")], ["SHLVL=1", "_=/usr/bin/strace", "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin", "LANG=C.UTF-8", "PWD=/"]
RE_CONNECT_ADDR = '(.*?),.*family=(.*),.*htons\(([0-9]*)\).*addr\(\"(.*)\"' # 4, {sa_family=(AF_INET), sin_port=htons(25000), sin_addr=inet_addr("123.129.217.153")}, 16
RE_CONNECT_FAMILY = 'family=(.*?),'
RE_LOG_MERGE_CALL = '^.*? (.*)' # 1874  (06:57:05.321833 brk(0)                  = 0x5dc000)
RE_IOCTL_NAME = 'ifr_name="(.*?)"'
RE_SENDTO_ADDR = '^(.*?),.*htons\((.*?)\).*addr\(\"(.*)\"\)' # (-1), "E\0\0(\", 40, MSG_NOSIGNAL, {sa_family=AF_INET, sin_port=htons(23), sin_addr=inet_addr("159.79.135.69")}, 16

ENDPOINT_REGEX = '([0-9]+\.*){4}:[0-9]+'
PRIVATE_IP_REGEX = '(^127\.)|(^10\.)|(^172\.1[6-9]\.)|(^172\.2[0-9]\.)|(^172\.3[0-1]\.)|(^192\.168\.)'

In [4]:
###### Syscall Table ######
process_creation = ['clone', 'fork', 'vfork']
open_file = ['open', 'openat']
write_file = ['write', 'rename']
info = ['uname', 'sysinfo']
send = ['send', 'sendto', 'sendmsg']
recv = ['recv', 'recvfrom', 'recvmsg']
remove = ['unlink', 'unlinkat', 'rmdir']
link = ['symlink', 'symlinkat', 'link', 'linkat']

#### ignore path ####
ignore_paths = ['/dev/null', 'stdout', 'stderr']

# Trace Parser

In [5]:
def parse_line(log):
    signal = re.search('--- (SIG.*?) {(.*)}', log)
    term = re.search('[0-9]* \+\+\+ (.*) \+\+\+', log)
    
    info = {}
    
    if signal:
        info['type'] = 'sig'
        info['name'] = signal.group(1)
        info['args'] = signal.group(2)
    elif term:
        info['type'] = 'term'
        info['status'] = term.group(1)
    else: 
        info['type'] = 'sys'
        g = re.search(' (.*?)\((.*)\).*= (.*)', log)
        
        # uncompleted function caused by terminating execution
        if not g:
            return None
        
        info['name'] = g.group(1)
        info['args'] = g.group(2)
        info['return'] = g.group(3)
        info['success'] = True
        info['return_msg'] = None
        
        
        # return status: True for success, False for error (startswith E or ?)
        ret = info['return'].split(' ')
        if len(ret) > 1:
            info['return'] = ret[0]
            info['return_msg'] = ' '.join(ret[1:])
            
            # inspect first char of return description ()
            if ret[1].startswith('E'):
                info['success'] = False
                info['return_msg'] = ' '.join(ret[2:])
                
        if info['return'] == '?':
            info['success'] = False
    
    time = re.search(RE_TIME, log).group(1)
    info['time'] = time
    
    return info

In [6]:
def strace_log_merge(path):
    f = open(f'{path}/merge.log', 'w')
    print("Strace merge path:", path)
    p = subprocess.Popen(['strace-log-merge', f'{path}/strace.log'], stdout = f, bufsize=0)
    out, err = p.communicate()
    f.close()
    if err:
        return False
    
    return True

In [7]:
def myprint(syscall, name):
    if syscall['name'] == name:
        print(f"{syscall['args']}: {syscall['return']}")

In [8]:
class Node:
    def __init__(self, name, typ='p'):
        self.name = name
        self.type = typ
        self.technique = ''
        self.tactic = ''
        self.color = None
        self.id = None
class Edge:
    def __init__(self, name, timestamp):
        self.name = name
        self.timestamp = timestamp
        self.technique = ''
        self.tactic = ''
        self.color = None
        self.id = None

# File Descriptor Handler

In [9]:
class FileTable:
    def __init__(self):
        self.fd_table = {
            '0': 'stdin',
            '1': 'stdout',
            '2': 'stderr'
        }
    
    def is_used(self, fd):
        return fd in self.fd_table
    
    def add(self, fd, path = None):
        self.fd_table[fd] = path
        
    def rm(self, fd):
        if self.is_used(fd):
            return self.fd_table.pop(fd)
        else:
            return False
        
    def get(self, fd):
        if self.is_used(fd):
            return self.fd_table[fd]
        else:
            return False
        
    def update(self, fd, path):
        if self.is_used(fd):
            self.fd_table[fd] = path
            return True
        else:
            return False
        
    def __str__(self):
        display = '-----------\n'
        for fd, path in self.fd_table.items():
            display += f'{fd}: {path}\n'
        display += '-------------\n'
        return display

# System Call Parser

In [73]:
class SyscallParser:
    def __init__(self):        
        self.parse_handler = {
            'execve': self.execve,
            'open': self.openn,
            'openat': self.openat,
            'read': self.read,
            'close': self.close,
            'write': self.write,
            'rename': self.rename,
            'socket': self.socket,
            'connect': self.connect,
            'send': self.send,
            'sendto': self.sendto,
            'sendmsg': self.sendmsg,
            'recv': self.recv,
            'recvfrom': self.recvfrom,
            'recvmsg': self.recvmsg,
            'ioctl': self.ioctl,
            'unlink': self.unlink,
            'unlinkat': self.unlinkat,
            'rmdir': self.rmdir,
            'bind': self.bind,
            'mkdir': self.mkdir,
            'symlink': self.symlink,
            'symlinkat': self.symlinkat,
            'link': self.link,
            'linkat': self.linkat,
            'kill': self.kill,
            'ptrace': self.ptrace,

            ### dofloo v1 ###
            "readlink": self.readlink,
            "waitpid": self.waitpid,
            "access": self.access,
            "fstat": self.fstat,
            "mmap": self.mmap,
            "stat": self.stat,
            "wait4": self.wait4,
            "statfs": self.statfs,
            "fcntl": self.fcntl,
            "fchown": self.fchown,
            "fchmod": self.fchmod,
            "mmap2": self.mmap2,
            "_newselect": self.newselect,
            "getsockopt": self.getsockopt,
            "fstat64": self.fstat64,
            "lseek": self.lseek,

            ### dofloo v2 ###
            "brk": self.brk,
            "set_thread_area": self.set_thread_area,
            "set_tid_address": self.set_tid_address,
            "set_robust_list": self.set_robust_list,
            "futex": self.futex,
            "ugetrlimit": self.ugetrlimit,
            "getcwd": self.getcwd,
            "exit_group": self.exit_group,
            "mprotect": self.mprotect,
            "arch_prctl": self.arch_prctl,
            "munmap": self.munmap,
            "getuid": self.getuid,
            "getgid": self.getgid,
            "getpid": self.getpid,
            "geteuid": self.geteuid,
            "getppid": self.getppid,
            "getegid": self.getegid,
            "prlimit64": self.prlimit64,
            "umask": self.umask,
            "setsid": self.setsid,
            "nanosleep": self.nanosleep,
            "time": self.time,
            
            
            ### xorsddos ###
            "dup2": self.dup2,
            "stat64": self.stat64,
            "gettimeofday": self.gettimeofday,
            "shmget": self.shmget,
            "shmat": self.shmat,
            "shmdt": self.shmdt,
            "getdents": self.getdents,
            "lstat": self.lstat,
            "pipe": self.pipe,
            "fcntl64": self.fcntl64,
            "mremap": self.mremap,
            "newfstatat": self.newfstatat,
            "setsockopt": self.setsockopt,
            "getsockname": self.getsockname,
            "ppoll": self.ppoll,

            ### Tsunami ###
            "chdir": self.chdir
        }
        
        self.isMemSink = True

    def chdir(self):
        info = {}
        args  = self.syscall['args'].split(",")
        path = args[0]
        info["path"] = path
        
        return info
    
    def dup2(self):
        info = {}
        args  = self.syscall['args'].split(",")
        old_fd = args[0]
        new_fd = args[1].strip(" ")
        info["old_fd"] = old_fd
        info["new_fd"] = new_fd
        
        return info
    
    def stat64(self):
        info = {}
        args  = self.syscall['args'].split(",")
        path = args[0]
        info["path"] = path
        
        return info
    
    def gettimeofday(self):
        info = {}
        args  = self.syscall['args'].split(",")
        timestamp = int(args[0].split("=")[1])
        
        sys_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        # info["sys_time"] = sys_time
        info["sys_time"] = "Timestamp"
        
        return info
    
    def shmget(self):
        info = {}
        shm_id  = self.syscall['return']
        info["shm_id"] = "Shared Memory:" + shm_id
        
        return info
    
    def shmat(self):
        info = {}
        args  = self.syscall['args'].split(",")
        shm_id = args[0]
        shm_addr = self.syscall['return']
        info["shm_id"] = "Shared Memory:" + shm_id # 從 ID 指向 addr
        if self.isMemSink:
            info["shm_addr"] = "Memory Address"
        else:
            info["shm_addr"] = shm_addr
        
        return info
    
    def shmdt(self):
        info = {}
        args  = self.syscall['args'].split(",")
        shm_addr = args[0]
        
        if self.isMemSink:
            info["shm_addr"] = "Memory Address"
        else:
            info["shm_addr"] = shm_addr
        
        return info
    
    def getdents(self):
        info = {}
        args  = self.syscall['args'].split(",")
        fd = args[0]
        info["fd"] = fd
        
        return info
    
    def lstat(self):
        info = {}
        args  = self.syscall['args'].split(",")
        path = args[0]
        info["path"] = path
        
        return info
    
    def pipe(self):
        info = {}
        args  = self.syscall['args'].split(",")
        read_fd = re.findall(r"\d+", args[0])[0]
        write_fd = re.findall(r"\d+", args[1])[0]
        info["read_fd"] = read_fd
        info["write_fd"] = write_fd
        
        return info
    
    def fcntl64(self):
        info = {}
        args  = self.syscall['args'].split(",")
        fd = args[0]
        info["fd"] = fd
        
        return info
    
    def mremap(self):
        info = {}
        args  = self.syscall['args'].split(",")
        m_addr = args[0]

        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr
        return info
    
    def newfstatat(self):
        info = {}
        args  = self.syscall['args'].split(",")
        path = args[1]
        info["path"] = path
        
        return info
    
    def setsockopt(self):
        info = {}
        args  = self.syscall['args'].split(",")
        sck_id = args[0]
        info["sck_id"] = sck_id
        
        return info
    def getsockname(self):
        info = {}
        args  = self.syscall['args'].split(",")
        sck_id = args[0]
        info["sck_id"] = sck_id
        
        return info
    
    def ppoll(self):
        info = {}
        args  = self.syscall['args'].split(",")
        fds = re.findall( r"\d+", args[0] ) 
        info["fds"] = fds
        
        return info
    
    
    def brk(self):
        info = {}
        m_addr  = self.syscall['return']
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr

        return info
    
    def set_thread_area(self):
        info = {}
        args  = self.syscall['args']
        m_addr = args.split(",")[1].strip(" ").split("=")[1]
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr
        return info
    def set_tid_address(self):
        info = {}
        args  = self.syscall['args']
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = args.strip(" ")
        
        return info
    
    def set_robust_list(self):
        info = {}
        args  = self.syscall['args']
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = args.split(",")[0]
        
        return info
    
    def futex(self):
        info = {}
        args  = self.syscall['args']
          
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = args.split(",")[0]   
        
        return info
    
    def ugetrlimit(self):
        info = {}
        args  = self.syscall['args']
        res_limit = args.split(",")[1].strip(" ").split("=")[1]
        info["res_limit"] = res_limit + " bytes" # bytes
        
        return info
    def getcwd(self):
        info = {}
        args  = self.syscall['args']
        path = args.split(",")[0]
        info["path"] = path
        
        
        return info
    
    def exit_group(self):
        info = {}
        args  = self.syscall['args']
        status_code = args.split(",")[0]
        info["status_code"] = "status:"+ status_code
        
        
        return info
    def mprotect(self):
        info = {}
        args  = self.syscall['args']
        m_addr = args.split(",")[0]
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr
        
        
        return info
    
    def arch_prctl(self):
        info = {}
        args  = self.syscall['args']
        m_addr = args.split(",")[1].strip(" ")
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr
        
        return info
    def munmap(self):
        info = {}
        args  = self.syscall['args']
        m_addr = args.split(",")[0]
        
        if self.isMemSink:
            info["m_addr"] = "Memory Address"
        else:
            info["m_addr"] = m_addr
        
        
        return info
    
    def getuid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        
        return info
    
    def getgid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        
        
        return info
    
    def getpid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        
        return info
    def geteuid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        return info
    
    def getppid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        
        return info
    
    def getegid(self):
        info = {}
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"
        
        return info
    def prlimit64(self):
        info = {}
        args  = self.syscall['args']
        res_limit = args.split(",")[3].strip(" ").split("=")[1]
        info["res_limit"] = res_limit + " bytes"
        
        return info
    
    def umask(self):
        info = {}
        args  = self.syscall['args']
        permission_mask = args.split(",")[0]
        info["permission_mask"] = "Permission:" + permission_mask
        
        return info
    def setsid(self):
        info = {}
        args  = self.syscall['args']
        id  = self.syscall['return']
        # info["id"] = id
        info["id"] = "ID"        
        
        return info
    
    def nanosleep(self):
        info = {}
        args  = self.syscall['args']
        sec = args.split(",")[0].split("=")[1]
        nsec = args.split(",")[1].split("=")[1][0:-1]
        interval = sec + "s + " + nsec + "ns"
        
        # info["interval"] = interval
        info["interval"]  = "Sleep Duration"
        return info
    
    def time(self):
        info = {}
        timeStamp  = self.syscall['return']

        ts = int(timeStamp)

        # if you encounter a "year is out of range" error the timestamp
        # may be in milliseconds, try `ts /= 1000` in that case
        sys_time = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        # info["sys_time"] = sys_time
        info["sys_time"] = "Timestamp"
        
        return info
    
    def readlink(self):
        info = {}
        args  = self.syscall['args']
        path = args.split(",")[0].strip("\"")

        info["path"] = path
        
        return info
    
    def waitpid(self):
        info = {}
        args = self.syscall['args']
        pid = args.split(",")[0]
        info["pid"] = pid

        return info

    def access(self):
        info = {}
        args = self.syscall['args']
        path = args.split(",")[0].strip("\"")
        info["path"] = path
        
        return info
    
    def fstat(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        return info

    def mmap(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[-2].strip(" ")
        info["fd"] = fd
        return info
    
    def stat(self):
        info = {}
        args = self.syscall['args']
        path = args.split(",")[0].strip("\"")
        info["path"] = path
        return info

    def wait4(self):
        info = {}
        args = self.syscall['args']
        pid = args.split(",")[0]
        info["pid"] = pid
        return info

    def statfs(self):
        info = {}
        args = self.syscall['args']
        path = args.split(",")[0].strip("\"")
        info["path"] = path
        return info

    def fcntl(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        
        return info

    def fchown(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        return info

    def fchmod(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        return info

    def mmap2(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[-2]
        info["fd"] = fd
        return info

    def newselect(self):
        info = {}
        args = self.syscall['args']
        readfds_sets = re.findall(r'\d+', args.split(",")[1].strip(" "))
        writefds_sets = re.findall(r'\d+', args.split(",")[2].strip(" "))
        exceptfds_sets = re.findall(r'\d+', args.split(",")[3].strip(" "))
        info["readfds_sets"] = readfds_sets
        info["writefds_sets"] = writefds_sets
        info["exceptfds_sets"] = exceptfds_sets
        
        return info
        
    def getsockopt(self):
        info = {}
        args = self.syscall['args']
        sck_id = args.split(",")[0]
        info["sck_id"] = sck_id
        return info

    def fstat64(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        return info

    def lseek(self):
        info = {}
        args = self.syscall['args']
        fd = args.split(",")[0]
        info["fd"] = fd
        return info

    def execve(self):
        info = {}
        
        command = re.search(RE_EXECVE_COMMAND, self.syscall['args'])
        if command:
            command = command.group(1)
        else:
            command = re.search('\"(.*)\"', self.syscall['args'].split(',')[0]).group(1)
            
        option = re.search(RE_EXECVE_OPTION, self.syscall['args'])
        
        # if execve command has option: return option, else option = None
        if option:
            option = option.group(1)
            
        info['command'] = command
        info['option'] = option
        
        return info
    
    def openn(self):
        info = {}
        args = re.search('\"(.*)\", (.*)', self.syscall['args'])
        
        if not args:
            return False
        
        path = args.group(1)
        mode = args.group(2)
        
        info['path'] = path
        info['mode'] = mode
        
        return info
    
    def openat(self):
        info = {}
        args = re.search('\"(.*)\", (.*)', self.syscall['args'])
        
        if not args:
            return False
        
        path = args.group(1)
        mode = args.group(2)
        
        info['path'] = path
        info['mode'] = mode
        
        return info
    
    def close(self):
        info = {}
        info['fd'] = self.syscall['args']
        
        return info
    
    def write(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0]
        return info
    
    def rename(self):
        info = {}
        args = self.syscall['args'].split(',')
        info['path'] = re.search('\"(.*)\"', args[1]).group(1)
        
        return info
    
    def read(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0]
        return info
    
    def socket(self):
        info = {}
        info['fd'] = self.syscall['return']
        
        data = self.syscall['args'].split(',')
        info['family'] = data[0].strip()
        
        proto = data[1].strip()
        if proto == 'SOCK_STREAM':
            info['proto'] = 'TCP'
        elif proto == 'SOCK_DGRAM':
            info['proto'] = 'UDP'
        else:
            info['proto'] = proto
        
        return info
    
    def bind(self):
        info = {}
        family = re.search(RE_CONNECT_FAMILY, self.syscall['args']).group(1)
        if family not in ['AF_INET', 'PF_INET']:
            info['family'] = family
            return info
        
        conn = re.search(RE_CONNECT_ADDR, self.syscall['args'])
        
        info['fd'] = conn.group(1)
        info['family'] = conn.group(2)
        info['addr'] = conn.group(4) + ':' + conn.group(3)
        
        return info
    
    def connect(self):
        info = {}
        family = re.search(RE_CONNECT_FAMILY, self.syscall['args']).group(1)
        if family not in ['AF_INET', 'PF_INET']:
            info['family'] = family
            return info
        
        conn = re.search(RE_CONNECT_ADDR, self.syscall['args'])
        
        info['fd'] = conn.group(1)
        info['family'] = conn.group(2)
        info['addr'] = conn.group(4) + ':' + conn.group(3)
        
        return info
    
    def send(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0].strip()
        
        return info
    
    def sendto(self):
        # sendto(int sockfd, const void *buf, size_t len, int flags,
        #const struct sockaddr *dest_addr, socklen_t addrlen);
        info = {}
        send = re.search(RE_SENDTO_ADDR, self.syscall['args'])
        
        # sendto that have no addr info like AF_UNIX or dest_addr = NULL
        if not send:
            info['fd'] = self.syscall['args'].split(',')[0].strip()
            return info
        
        info['fd'] = send.group(1)
        info['addr'] = send.group(3) + ':' + send.group(2)
        
        return info
    
    def sendmsg(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0].strip()
        
        return info
    
    def recv(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0].strip()
        
        return info
    
    def recvfrom(self):
        info = {}
        recv = re.search(RE_SENDTO_ADDR, self.syscall['args'])
        
        # recvfrom that have no addr info like AF_UNIX or dest_addr = NULL
        if not recv:
            info['fd'] = self.syscall['args'].split(',')[0].strip()
            return info
        
        info['fd'] = recv.group(1)
        info['addr'] = recv.group(3) + ':' + recv.group(2)
        
        return info
    
    def recvmsg(self):
        info = {}
        info['fd'] = self.syscall['args'].split(',')[0].strip()
        
        return info
    
    def ioctl(self):
        info = {}
        #ioctl(0, SIOCGIFADDR, {ifr_name="eth0", ifr_addr={AF_INET, inet_addr("10.0.2.15")}})
        data = self.syscall['args'].split(',')
        info['fd'] = data[0].strip()
        info['req'] = data[1].strip()
        
        if info['req'] == 'SIOCGIFADDR':
            # info['dev'] = re.search(RE_IOCTL_NAME, self.syscall['args']).group(1)
            info['dev'] = "NIC"
            
        return info
    
    def unlink(self):
        info = {}
        info['path'] = re.search('\"(.*)\"', self.syscall['args']).group(1)
        
        return info
    
    def unlinkat(self):
        info = {}
        info['path'] = re.search('\"(.*)\"', self.syscall['args'].split(',')[1]).group(1)
        
        return info
    
    def symlink(self):
        # int symlink(const char *target, const char *linkpath);
        info = {}
        t, l = self.syscall['args'].split(',')
        info['target'] = re.search('\"(.*)\"', t).group(1)
        info['link'] = re.search('\"(.*)\"', l).group(1)
        
        return info
        
    def symlinkat(self):
        # int symlinkat(const char *target, int newdirfd, const char *linkpath);
        info = {}
        t, f, l = self.syscall['args'].split(',')
        info['target'] = re.search('\"(.*)\"', t).group(1)
        info['link'] = re.search('\"(.*)\"', l).group(1)
        
        return info
    
    def link(self):
        # int link(const char *oldpath, const char *newpath);
        info = {}
        t, l = self.syscall['args'].split(',')
        info['target'] = re.search('\"(.*)\"', t).group(1)
        info['link'] = re.search('\"(.*)\"', l).group(1)
        
        return info
    
    def linkat(self):
        # int linkat(int olddirfd, const char *oldpath, int newdirfd, const char *newpath, int flags);
        info = {}
        d = self.syscall['args'].split(',')
        info['target'] = re.search('\"(.*)\"', d[1]).group(1)
        info['link'] = re.search('\"(.*)\"', d[3]).group(1)
        
        return info
    
    def rmdir(self):
        # int rmdir(const char *pathname);
        info = {}
        info['path'] = re.search('\"(.*)\"', self.syscall['args']).group(1)
        
        return info
    
    def mkdir(self):
        # int mkdir(const char *pathname, mode_t mode); 
        info = {}
        info['path'] = re.search('\"(.*)\"', self.syscall['args'].split(',')[0]).group(1)
        
        return info
    
    def kill(self):
        #int kill(pid_t pid, int sig);
        info = {}
        info['pid'] = self.syscall['args'].split(',')[0]
        
        return info
    
    def ptrace(self):
        info = {}
        args = self.syscall['args'].split(',')
        info['req'] = args[0]
        
        if len(args) >= 2:
            info['pid'] = args[1].strip()
        
        return info
        
    def parse(self, syscall): # syscall is a dict returned by parse_line()
        self.syscall = syscall
        self.name = syscall['name']
        
        return self.parse_handler[self.name]()
    
    
sys_parser = SyscallParser()

# Attack Scenario Graph Main Class Definition

In [74]:
class AttackGraph:
    def __init__(self, path):
        self.malware_hash = path.split('/')[-1]
        self.traces = sorted(glob.glob(path + '/strace*'))
        self.proc_id  = -1 # initialize
        self.traces_lines = 0
        self.trigger_lines = 0
        self.graph = {}
        self.edges = {}
        self.current_dir = ""
        self.isDuplicate = True

        self.proc_node_map = {}
        self.file_table = FileTable()
        
        # record seen file path
        self._file_node_map = {}
        
        # unique ip set
        self.unique_ip = set()
        
        # handler function
        self._syscall_handler_table = {
            'process': self._process,
            'read': self._read,
            'open': self._openn,
            'execve': self._execve,
            'close': self._close,
            'write': self._write,
            'socket': self._socket,
            'connect': self._connect,
            'info': self._info,
            'send': self._send,
            'recv': self._recv,
            'ioctl': self._ioctl,
            'rm': self._rm,
            'bind': self._bind,
            'mkdir': self._mkdir,
            'link': self._link,
            'kill': self._kill,
            'ptrace': self._ptrace
        }

        self.added_handler_table = [
            ### Dofloo v1 ###
            "readlink",
            "waitpid",
            "access",
            "fstat",
            "mmap",
            "stat",
            "wait4",
            "statfs",
            "fcntl",
            "fchown",
            "fchmod",
            "mmap2",
            "_newselect",
            "getsockopt",
            "fstat64",
            "lseek",
            
            ### Dofloo v2 ###
            "brk",
            "set_thread_area",
            "set_tid_address",
            "set_robust_list",
            "futex",
            "ugetrlimit",
            "getcwd",
            "exit_group",
            "mprotect",
            "arch_prctl",
            "munmap",
            "getuid",
            "getgid",
            "getpid",
            "geteuid",
            "getppid",
            "getegid",
            "prlimit64",
            "umask",
            "setsid",
            "nanosleep",
            "time",
            
            ### xorddos ###
            "dup2",
            "stat64",
            "gettimeofday",
            "shmget",
            "shmat",
            "shmdt",
            "getdents",
            "lstat",
            "pipe",
            "fcntl64",
            "mremap",
            "newfstatat",
            "setsockopt",
            "getsockname",
            "ppoll",

            ### Tsunami ###
            "chdir"
        ]
        
        # self.added_handler_table = {
        #     "readlink": self._readlink,
        #     "waitpid": self._waitpid,
        #     "access": self._access,
        #     "fstat": self._fstat,
        #     "mmap": self._mmap,
        #     "stat": self._stat,
        #     "wait4": self._wait4,
        #     "statfs": self._statfs,
        #     "fcntl": self._fcntl,
        #     "fchown": self._fchown,
        #     "fchmod": self._fchmod,
        #     "mmap2": self._mmap2,
        #     "_newselect": self._newselect,
        #     "getsockopt": self._getsockopt,
        #     "fstat64": self._fstat64,
        #     "lseek": self._lseek
        # }
        
        self._path = path
        self.exec = False
        self.step_list = []
        
    def replace_cwd(self, node):
        cwd = '/prober/host_share/'
        if node.name.split('/')[-1] == self.malware_hash:
            node.name = 'malware'
    
        if node.name.startswith(cwd):
            node.name = './' + node.name.replace(cwd, '')
    
    # connect two nodes
    def _connect_node(self, from_node, to_node, edge):
        #if to_node.name == 'sysinfo':
        #    print(f'Connect {from_node.name} to {to_node.name}')
        self.replace_cwd(from_node)
        self.replace_cwd(to_node)
        
        self.graph[from_node].append(to_node)
        self.edges[(from_node, to_node, edge.name)] = edge
        self.step_list.append((from_node, to_node, edge.name))
    
    # connect proc to file if there is no edge between them currently
    # from_proc: edge direction
    # if True: current node to file node (write, exec...). False: file node to current node (read)
    def _connect_proc_file(self, path, edge_name, time, from_proc = True, node_typ = 'f'):
        
        # if file node not exist: create new node, else: connect current node to exist file node
        if path not in self._file_node_map and path not in self.proc_node_map:
            child_node = Node(path, node_typ)
            
            if from_proc:
                from_node = self.current_node
                to_node = child_node
            else:
                from_node = child_node
                to_node = self.current_node
            
            if node_typ != 'p':
                self._file_node_map[path] = child_node
            else:
                self.proc_node_map[path] = child_node
            
            self.graph[child_node] = []
            self._connect_node(from_node, to_node, Edge(edge_name, time)) 
        # file node exist
        else:
            if node_typ != 'p':
                child_node = self._file_node_map[path]
            else:
                child_node = self.proc_node_map[path]
                
            if from_proc:
                from_node = self.current_node
                to_node = child_node
            else:
                from_node = child_node
                to_node = self.current_node
                
            # if curent node has not connected to file node (escape loop in strace.log)
            if not self.isDuplicate :
                if (from_node, to_node, edge_name) not in self.edges:
                    self._connect_node(from_node, to_node, Edge(edge_name, time))
            else:
                self._connect_node(from_node, to_node, Edge(edge_name, time))
    
    # remove connection between two nodes
    def _rm_connect(self, edge):
        from_node, to_node, _ = edge
        self.graph[from_node].remove(to_node)
        self.edges.pop(edge)
        
    def _find_path_by_fd(self, fd, time):
        path = 'Unknown'
        local_file_table = FileTable()
        
        # malware use only one ip
        if len(self.unique_ip) == 1:
            path = list(self.unique_ip)[0]
        
        # rebuild fd table from first line of merge.log
        else:
            if not os.path.isfile(f'{self._path}/merge.log'):
                strace_log_merge(self._path)

            with open(f'{self._path}/merge.log', 'r') as f:
                for line in f:
                    try:
                        line2 = re.search(RE_LOG_MERGE_CALL, line).group(1)
                    except AttributeError:
                        print(line)
                        raise
                    syscall = parse_line(line2)

                    if not syscall or syscall['type'] != 'sys':
                        continue

                    name = syscall['name']

                    # if found: get path and update class file_table
                    if syscall['time'] == time:
                        path = local_file_table.get(fd)
                        if not path:
                            pass
                            #print(syscall)
                            #raise KeyError('_find_path_by_fd: fd not found.')
                        self.file_table.update(fd, path)
                        break

                    # build fd table
                    if syscall['success']:
                        if name in open_file:
                            info = sys_parser.parse(syscall)
                            local_file_table.add(syscall['return'], info['path'])

                            # print("syscall:", syscall)
                            # print(local_file_table)
                        elif name == 'socket':
                            info = sys_parser.parse(syscall)
                            # update fd table if create socket successfully
                            if info['family'] not in ['AF_INET', 'PF_IENT']:
                                local_file_table.add(fd, info['family'])
                            else:
                                local_file_table.add(syscall['return'])

                        elif name == 'close':
                            info = sys_parser.parse(syscall)
                            local_file_table.rm(info['fd'])
                            
                    elif name == 'connect':
                        info = sys_parser.parse(syscall)
                        # ignore IPC usage
                        if info['family'] not in ['AF_INET', 'PF_INET']:
                            return
                        local_file_table.update(info['fd'], info['addr'])
        
        
        return path
        
    def _process(self, syscall):
        pid = syscall['return']
        child_node = Node(pid)
        self.proc_node_map[pid] = child_node
        self.graph[child_node] = []

        # if self._inside_execve:
        if self.exec:
            edge = Edge('exec', syscall['time'])
            
        else:
            edge = Edge(syscall['name'], syscall['time'])

        self._connect_node(self.current_node, child_node, edge)
    
    def _execve(self, syscall):
#         self._inside_execve = True # True will not draw the system file
        self.exec = True # Flag for present exec edge
        info = sys_parser.parse(syscall)
        if info['command'].split('/')[-1] == self.malware_hash:
            info['command'] = 'malware'

        # set current ndoe name = execve command
        self.current_node.name = info['command']

        # find target file if command has option
        if info['option']:
            self._execve_options = [ re.search('\"(.*)\"', opt).group(1) for opt in info['option'].split(',')]
        else:
            self._execve_options = []

    def _added_handler(self, syscall):
        info = sys_parser.parse(syscall)
        # for file
        if("fd" in info or "path" in info):
            
            if("fd" in info):
                fd = info['fd']
                path = self.file_table.get(fd)
                if not path and not self._inside_execve:
                    path = self._find_path_by_fd(fd, syscall['time'])
                    
                if path in ignore_paths:
                    return
            elif("path" in info):
                path = info["path"]

            
            edge_name = syscall["name"]

            

            # if fd exist in fd table 
            if path:
                if path[0] != "/":
                    path = self.current_dir + path

                # if file appear in execve or currently not in execve
                if (self._inside_execve and (path in self._execve_options)) or not self._inside_execve:
                    # file node not exist
                    if path not in self._file_node_map:
                        child_node = Node(path, typ='f')
                        self._file_node_map[path] = child_node
                        self.graph[child_node] = []
                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))


                    # file node exist
                    else:
                        child_node = self._file_node_map[path]
                        
                        # if curent node has connected to file node open(), remove open and connect edge
                        replace_edge_name = ['open']
                        for exist_edge_name in replace_edge_name:
                            edge = (self.current_node, child_node, exist_edge_name)
                            if edge in self.edges:
                                self._rm_connect(edge)
                        
                        # if has conected with write(), don't re-connect current node and child node
                        if not self.isDuplicate :
                            if (self.current_node, child_node, edge_name) not in self.edges:
                                self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                        else:
                            self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))

            ### For chdir ###
            if syscall["name"] == "chdir":
                self.current_dir = info["path"].strip("\"") + "/"

        # For process
        elif("pid" in info):
            
            pid = info["pid"]
            edge_name = syscall["name"]
 
            if pid in self.proc_node_map: # the process id need to have alreay been create, then we just can manipulate it
                child_node = self.proc_node_map[pid]
                
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
            else:
                node_name = "NO_PID"
                if node_name not in self._file_node_map:
                    child_node = Node(node_name)
                    self._file_node_map[node_name] = child_node
                    self.graph[child_node] = []
                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                else:
                    child_node = self._file_node_map[node_name]
                    if not self.isDuplicate :
                        if (child_node, self.current_node, edge_name) not in self.edges:
                            self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                    else:
                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                    
        # For socket
        elif("sck_id" in info):
            sck_id = info["sck_id"]
            path = self._find_path_by_fd(sck_id, syscall['time']) # get socket ip addr
            edge_name = syscall["name"]

            # file node not exist
            if path not in self._file_node_map:
                child_node = Node(path, typ='n')
                self._file_node_map[path] = child_node
                self.graph[child_node] = []
                self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))

            # file node exist
            else:
                child_node = self._file_node_map[path]
                # if curent node has connected to file node open(), remove open edge
                # replace_edge_name = ['connect']
                # for exist_edge_name in replace_edge_name:
                #     edge = (self.current_node, child_node, exist_edge_name)
                #     if edge in self.edges:
                #         self._rm_connect(edge)

                # # if has conected with recv(), don't re-connect current node and child node
                # if (child_node, self.current_node, edge_name) not in self.edges:
                # if (self.current_node, child_node, edge_name) not in self.edges:
                self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
                
        ### non-classify type ###
        elif("m_addr" in info):
            # return
            m_addr = info["m_addr"]
            edge_name = syscall["name"]

            # if "Memory Addr" not in self._file_node_map:
            #     child_node = Node("Memory Addr", typ='m_addr')
            #     self._file_node_map["Memory Addr"] = child_node
            #     self.graph[child_node] = []
            #     # if not self._inside_execve:
            #     self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))                
            # else:
            #     child_node = self._file_node_map["Memory Addr"]
            #     if (self.current_node, child_node, edge_name) not in self.edges:
            #         self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))  

            # node not exist
            if m_addr not in self._file_node_map:
                child_node = Node(m_addr, typ='m_addr')
                self._file_node_map[m_addr] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     self._connect_node(self.proc_node_map[self.proc_id], child_node,  Edge(edge_name, syscall['time']))
                    
            # node exist
            else:
                child_node = self._file_node_map[m_addr]
                # if not self._inside_execve:  
                # don't re-connect current node and child node   
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                        
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                    
                # else:
                #     # don't re-connect current node and child node
                #     if (self.proc_node_map[self.proc_id], child_node, edge_name) not in self.edges:
                #         self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
                
            
        elif("res_limit" in info):
            res_limit = info["res_limit"]
            edge_name = syscall["name"]
            # node not exist
            if res_limit not in self._file_node_map:
                child_node = Node(res_limit, typ='else')
                self._file_node_map[res_limit] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
            # node exist
            else:
                child_node = self._file_node_map[res_limit]
                # if not self._inside_execve:  
                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                    
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                    
                # else:
                #     # don't re-connect current node and child node
                #     if (self.proc_node_map[self.proc_id], child_node, edge_name) not in self.edges:
                #         self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))            
            
        elif("status_code" in info):
            status_code = info["status_code"]
            edge_name = syscall["name"]
            # node not exist
            if status_code not in self._file_node_map:
                child_node = Node(status_code, typ='else')
                self._file_node_map[status_code] = child_node
                self.graph[child_node] = []
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))

            # node exist
            else:
                child_node = self._file_node_map[status_code]

                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
  
        elif("id" in info):
            id = info["id"]
            edge_name = syscall["name"]
            # node not exist
            if id not in self._file_node_map:
                child_node = Node(id, typ='else')
                self._file_node_map[id] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                    # self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
            # node exist
            else:
                child_node = self._file_node_map[id]
                # if not self._inside_execve:  
                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
  

        elif("permission_mask" in info):
            permission_mask = info["permission_mask"]
            edge_name = syscall["name"]
            # node not exist
            if permission_mask not in self._file_node_map:
                child_node = Node(permission_mask, typ='else')
                self._file_node_map[permission_mask] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
            # node exist
            else:
                child_node = self._file_node_map[permission_mask]
                # if not self._inside_execve:  
                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     # don't re-connect current node and child node
                #     if (self.proc_node_map[self.proc_id], child_node, edge_name) not in self.edges:
                #         self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))   
        elif("interval" in info):
            interval = info["interval"]
            edge_name = syscall["name"]
            # node not exist
            if interval not in self._file_node_map:
                child_node = Node(interval, typ='else')
                self._file_node_map[interval] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
            # node exist
            else:
                child_node = self._file_node_map[interval]
                # if not self._inside_execve:  
                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     # don't re-connect current node and child node
                #     if (self.proc_node_map[self.proc_id], child_node, edge_name) not in self.edges:
                #         self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time'])) 
        elif("sys_time" in info):
            sys_time = info["sys_time"]
            edge_name = syscall["name"]
            # node not exist
            if sys_time not in self._file_node_map:
                child_node = Node(sys_time, typ='else')
                self._file_node_map[sys_time] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time']))
            # node exist
            else:
                child_node = self._file_node_map[sys_time]
                # if not self._inside_execve:  
                # don't re-connect current node and child node
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                     self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                # else:
                #     # don't re-connect current node and child node
                #     if (self.proc_node_map[self.proc_id], child_node, edge_name) not in self.edges:
                #         self._connect_node(self.proc_node_map[self.proc_id] , child_node,  Edge(edge_name, syscall['time'])) 
        elif("old_fd" in info and "new_fd" in info):
            old_fd = info["old_fd"]
            new_fd = info["new_fd"]   
            self.file_table.update( new_fd, self.file_table.get(old_fd) )

        elif("shm_id" in info):
            shm_id = info["shm_id"]
            edge_name = syscall["name"]
            
            if ("shm_addr" not in info):
                if shm_id not in self._file_node_map:
                    child_node = Node(shm_id, typ='else')
                    self._file_node_map[shm_id] = child_node
                    self.graph[child_node] = []
                    # if not self._inside_execve:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    child_node = self._file_node_map[shm_id]
                    if not self.isDuplicate :
                        if (self.current_node, child_node, edge_name) not in self.edges:
                            self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                    else:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                    
            else:
                # return 
                shm_addr = info["shm_addr"]

                # if "Memory Addr" not in self._file_node_map:
                #     child_node = Node("Memory Addr", typ='m_addr')
                #     self._file_node_map["Memory Addr"] = child_node
                #     self.graph[child_node] = []
                #     # if not self._inside_execve:
                #     self._connect_node(self._file_node_map[shm_id] , child_node,  Edge(edge_name, syscall['time']))                  
                # else:
                #     child_node = self._file_node_map["Memory Addr"]
                #     if (self.current_node, child_node, edge_name) not in self.edges:
                #         self._connect_node(self._file_node_map[shm_id] , child_node,  Edge(edge_name, syscall['time']))   

                # not have the addr
                if shm_addr not in self._file_node_map:

                    child_node = Node(shm_addr, typ='m_addr')
                    self._file_node_map[shm_addr] = child_node
                    self.graph[child_node] = []
                    # if not self._inside_execve:
                    self._connect_node(self._file_node_map[shm_id] , child_node,  Edge(edge_name, syscall['time']))
                    
                # if have addr, the id connect to the addr    
                else:
                    child_node = self._file_node_map[shm_addr]
                    if not self.isDuplicate :
                        if (self.current_node, child_node, edge_name) not in self.edges:
                            self._connect_node(self._file_node_map[shm_id] , child_node,  Edge(edge_name, syscall['time']))
                    else:
                        self._connect_node(self._file_node_map[shm_id] , child_node,  Edge(edge_name, syscall['time']))
                        
                        
        elif("read_fd" in info and "write_fd" in info):
            read_fd = info["read_fd"]
            write_fd = info["write_fd"]
            edge_name = syscall["name"]
            
            read_node = "Pipe " + read_fd + " : read end from Pipe " + write_fd
            write_node = "Pipe " + write_fd + " : write end to Pipe " + read_fd
            
            self.file_table.add(read_fd, read_node)
            self.file_table.add(write_fd, write_node)
            
            if read_node not in self._file_node_map:
                child_node = Node(read_node, typ='pipe')
                self._file_node_map[read_node] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
            else:
                child_node = self._file_node_map[read_node]
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
            
            if write_node not in self._file_node_map:
                child_node = Node(write_node, typ='pipe')
                self._file_node_map[write_node] = child_node
                self.graph[child_node] = []
                # if not self._inside_execve:
                self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))
            else:
                child_node = self._file_node_map[write_node]
                if not self.isDuplicate :
                    if (self.current_node, child_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))    
                else:
                    self._connect_node(self.current_node , child_node,  Edge(edge_name, syscall['time']))  
                    
        elif("fds" in info or "readfds_sets" in info) :
            if "fds" in info:
                fds = info["fds"]
                edge_name = syscall["name"]
                for fd in fds:
                    path = self.file_table.get(fd)
                    if not path and not self._inside_execve:
                        path = self._find_path_by_fd(fd, syscall['time'])
                    if path in ignore_paths:
                        continue
                    
                    # if fd exist in fd table 
                    if path:
                        if path[0] != "/":
                            path = self.current_dir + path
                        # if file appear in execve or currently not in execve
                        if (self._inside_execve and (path in self._execve_options)) or not self._inside_execve:
                            
                            # file node not exist
                            if path not in self._file_node_map:
                                child_node = Node(path, typ='f')
                                self._file_node_map[path] = child_node
                                self.graph[child_node] = []
                                self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))


                            # file node exist
                            else:
                                child_node = self._file_node_map[path]
                                
                                # if curent node has connected to file node open(), remove open and connect edge
                                replace_edge_name = ['open']
                                for exist_edge_name in replace_edge_name:
                                    edge = (self.current_node, child_node, exist_edge_name)
                                    if edge in self.edges:
                                        self._rm_connect(edge)

                                # if has conected with write(), don't re-connect current node and child node
                                if not self.isDuplicate :
                                    if (self.current_node, child_node, edge_name) not in self.edges:
                                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                                else:
                                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                                    
            else:
                for key in info:
                    if len(info[key]) != 0:

                        fds = info[key]
                        edge_name = syscall["name"]
                        for fd in fds:
                            path = self.file_table.get(fd)
                            if not path and not self._inside_execve:
                                path = self._find_path_by_fd(fd, syscall['time'])
                            if path in ignore_paths:
                                continue
                            
                            # if fd exist in fd table 
                            if path:
                                if path[0] != "/":
                                    path = self.current_dir + path
                                # if file appear in execve or currently not in execve
                                if (self._inside_execve and (path in self._execve_options)) or not self._inside_execve:
                                    
                                    # file node not exist
                                    if path not in self._file_node_map:
                                        child_node = Node(path, typ='f')
                                        self._file_node_map[path] = child_node
                                        self.graph[child_node] = []
                                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))


                                    # file node exist
                                    else:
                                        child_node = self._file_node_map[path]
                                        
                                        # if curent node has connected to file node open(), remove open and connect edge
                                        replace_edge_name = ['open']
                                        for exist_edge_name in replace_edge_name:
                                            edge = (self.current_node, child_node, exist_edge_name)
                                            if edge in self.edges:
                                                self._rm_connect(edge)
                                        
                                        # if has conected with write(), don't re-connect current node and child node
                                        if not self.isDuplicate :
                                            if (self.current_node, child_node, edge_name) not in self.edges:
                                                self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                                        else:
                                            self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
        
    def _read(self, syscall):
        info = sys_parser.parse(syscall)
        fd = info['fd']
        path = self.file_table.get(fd)
        
        # fd may open in parent process
        if not path and not self._inside_execve:
            path = self._find_path_by_fd(fd, syscall['time'])
            
        # ignore paths
        if path in ignore_paths:
            return
        
        edge_name = 'read'
#         if not syscall['success']:
#             edge_name = f"read{syscall['return_msg']}"
        
        # if fd exist in fd table 
        if path:
            
            ### For pipe ###
            if("Pipe" in path):
                read_fd = path.split(" ")[1]
                write_fd= path.split(" ")[-1]
                
                read_node = "Pipe " + read_fd + " : read end from Pipe " + write_fd
                write_node = "Pipe " + write_fd + " : write end to Pipe " + read_fd
                
                # 不重複畫 edge
                if not self.isDuplicate :
                    if (self._file_node_map[write_node],  self._file_node_map[read_node], edge_name) not in self.edges:
                        self._connect_node(self._file_node_map[write_node] , self._file_node_map[read_node],  Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self._file_node_map[write_node] , self._file_node_map[read_node],  Edge(edge_name, syscall['time']))
            if path[0] != "/":
                path = self.current_dir + path      
            # if file appear in execve or currently not in execve
            if (self._inside_execve and (path in self._execve_options)) or not self._inside_execve:
                # file node not exist
                if path not in self._file_node_map:
                    child_node = Node(path, typ='f')
                    self._file_node_map[path] = child_node
                    self.graph[child_node] = []
                    self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))

                # file node exist
                else:
                    child_node = self._file_node_map[path]
                    # if curent node has connected to file node open(), remove open edge
                    replace_edge_name = ['open', 'connect']
                    for exist_edge_name in replace_edge_name:
                        edge = (self.current_node, child_node, exist_edge_name)
                        if edge in self.edges:
                            self._rm_connect(edge)

                    # if has conected with read(), don't re-connect current node and child node
                    if not self.isDuplicate :
                        if (child_node ,self.current_node, edge_name) not in self.edges:
                            self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
                    else:
                        self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
                    
            
    def _write(self, syscall):
        info = sys_parser.parse(syscall)
        if 'fd' in info:
            fd = info['fd']
            path = self.file_table.get(fd)
        else:
            path = info['path']
        
        # fd may open in parent process
        if not path and not self._inside_execve:
            path = self._find_path_by_fd(fd, syscall['time'])
        
        # ignore paths
        if path in ignore_paths:
            return
            
        edge_name = 'write'
#         if not syscall['success']:
#             edge_name = f"write{syscall['return_msg']}"
        
        # if fd exist in fd table 
        if path:
           
            ### For pipe ###
            if("Pipe" in path):
                read_fd = path.split(" ")[-1]
                write_fd= path.split(" ")[1]
                
                read_node = "Pipe " + read_fd + " : read end from Pipe " + write_fd
                write_node = "Pipe " + write_fd + " : write end to Pipe " + read_fd
                
                # 不重複畫 edge
                if not self.isDuplicate :
                    if (self._file_node_map[write_node],  self._file_node_map[read_node], edge_name) not in self.edges:
                        self._connect_node(self._file_node_map[write_node] , self._file_node_map[read_node],  Edge(edge_name, syscall['time']))      
                else:
                    self._connect_node(self._file_node_map[write_node] , self._file_node_map[read_node],  Edge(edge_name, syscall['time'])) 
            if path[0] != "/":
                path = self.current_dir + path
            # if file appear in execve or currently not in execve
            if (self._inside_execve and (path in self._execve_options)) or not self._inside_execve:
                # file node not exist
                if path not in self._file_node_map:
                    child_node = Node(path, typ='f')
                    self._file_node_map[path] = child_node
                    self.graph[child_node] = []
                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))

                # file node exist
                else:
                    child_node = self._file_node_map[path]
                    
                    # if curent node has connected to file node open(), remove open and connect edge
                    replace_edge_name = ['open', 'connect']
                    for exist_edge_name in replace_edge_name:
                        edge = (self.current_node, child_node, exist_edge_name)
                        if edge in self.edges:
                            self._rm_connect(edge)
                    
                    # if has conected with write(), don't re-connect current node and child node
                    if not self.isDuplicate :
                        if (self.current_node, child_node, edge_name) not in self.edges:
                            self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                    else:
                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
           
    def _openn(self, syscall):
        info = sys_parser.parse(syscall)
        if not info:
            return
        path = info['path']
        if path[0] != "/":
            path = self.current_dir + path
        edge_name = 'open'

#         if not syscall['success']:
#             edge_name = f"open{syscall['return_msg']}"

        # update fd table if open successfully
        if syscall['success']:
            self.file_table.add(syscall['return'], info['path'])
            #print(f"open {info['path']}: {self.file_table}")
        
        if path in ignore_paths:
            return
        
        # open and openat appear after execve in same strace.log
        if self._inside_execve:
            # find path that apeears in execve command option
            #print(path, self._execve_options, self._execve_options[0])
            if path in self._execve_options:
                self._connect_proc_file(path, edge_name, syscall['time']) 

        # not inside execve
        else:
            self._connect_proc_file(path, edge_name, syscall['time'])
        
    def _close(self, syscall):
        info = sys_parser.parse(syscall)
        fd = info['fd']
        
        # delete fd only if successfully close
        if syscall['success']:
            self.file_table.rm(fd)
        
    def _info(self, syscall):
        edge_name = 'read'
        path = syscall['name']
        if path[0] != "/":
            path = self.current_dir + path
        if not self._inside_execve:
            self._connect_proc_file(path, edge_name, syscall['time'], from_proc = False, node_typ = 'c')

        
    def _socket(self, syscall):
        info = sys_parser.parse(syscall)
        fd = info['fd']
        
        if not syscall['success']:
            return
        
        # update fd table if create socket successfully
        if info['family'] not in ['AF_INET', 'PF_INET']:
            self.file_table.add(fd, "NOT_NET")
        else:
            self.file_table.add(syscall['return'])
        #print('socket: ', self.file_table)
        
    def _bind(self, syscall):
        info = sys_parser.parse(syscall)
        
        # ignore IPC usage
        if info['family'] not in ['AF_INET', 'PF_INET']:
            return
        
        fd = info['fd']
        path = info['addr']
        edge_name = 'bind'
        
        res = self.file_table.update(fd, path)
        if not res:
            print('socket fd not found.')
            raise KeyError
            
        self._connect_proc_file(path, edge_name, syscall['time'], node_typ = 'n')
        
    def _connect(self, syscall):
        info = sys_parser.parse(syscall)
        
        # ignore IPC usage
        if info['family'] not in ['AF_INET', 'PF_INET']:
            return
        
        fd = info['fd']
        path = info['addr']
        self.unique_ip.add(path)
        edge_name = 'connect'
        
        res = self.file_table.update(fd, path)
        if not res:
            print('socket fd not found.')
            raise KeyError
        
        # if file node not exist: create new node, else: connect current node to exist file node
        self._connect_proc_file(path, edge_name, syscall['time'], node_typ = 'n')
    
    def _recv(self, syscall):
        info = sys_parser.parse(syscall)
        fd = info['fd']
        edge_name = 'recv'

        if fd == "-1":
            node_name = "NO_SOCKET"
            if node_name not in self._file_node_map:
                child_node = Node(node_name, typ='n')
                self._file_node_map[node_name] = child_node
                self.graph[child_node] = []
                self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
            else:
                child_node = self._file_node_map[node_name]
                if not self.isDuplicate :
                    if (child_node, self.current_node, edge_name) not in self.edges:
                        self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))      
            return 
        # socket error
        # if fd == '-1':
        #     # get ip addr from recvfrom
        #     if syscall['name'] == 'recvfrom':
        #         if 'addr' in info:
        #             path = info['addr']
        #             self.unique_ip.add(path)
        #         elif len(self.unique_ip) == 1:
        #             path = list(self.unique_ip)[0]
        #         else:
        #             return
                    
        #     else:
        #         return
            
        # socket create successfully
        if fd != '-1':
            # AF_UNIX socket
            if self.file_table.get(fd) == "NOT_NET":
                return
            
            # if malware use UDP, we need to map fd to ip addr through sendto
            if syscall['name'] == 'recvfrom' and not self.file_table.get(fd):
                if 'addr' in info:
                    path = info['addr']

                elif len(self.unique_ip) == 1:
                    path = list(self.unique_ip)[0]

                self.file_table.update(fd, path)
                self.unique_ip.add(path)       
                
            path = self.file_table.get(fd)
            # created in parent process, find it
            if not path:
                path = self._find_path_by_fd(fd, syscall['time'])
        
        # file node not exist
        if path not in self._file_node_map:
            child_node = Node(path, typ='n')
            self._file_node_map[path] = child_node
            self.graph[child_node] = []
            self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))

        # file node exist
        else:
            child_node = self._file_node_map[path]
            # if curent node has connected to file node open(), remove open edge
            replace_edge_name = ['connect']
            for exist_edge_name in replace_edge_name:
                edge = (self.current_node, child_node, exist_edge_name)
                if edge in self.edges:
                    self._rm_connect(edge)

            # if has conected with recv(), don't re-connect current node and child node
            if not self.isDuplicate :
                if (child_node, self.current_node, edge_name) not in self.edges:
                    self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
            else:
                self._connect_node(child_node, self.current_node, Edge(edge_name, syscall['time']))
                

                        
    def _send(self, syscall):
        info = sys_parser.parse(syscall)
        fd = info['fd']
        edge_name = 'send'

        if fd == "-1":
            node_name = "NO_SOCKET"
            if node_name not in self._file_node_map:
                child_node = Node(node_name, typ='n')
                self._file_node_map[node_name] = child_node
                self.graph[child_node] = []
                self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
            else:
                child_node = self._file_node_map[node_name]
                if not self.isDuplicate :
                    if (child_node, self.current_node, edge_name) not in self.edges:
                        self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                else:
                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))

            return       
        # socket create unsuccessfully
        # if fd == '-1':
        #     if syscall['name'] == 'sendto':
        #         if 'addr' in info:
        #             path = info['addr']
        #             self.unique_ip.add(path)
        #         elif len(self.unique_ip) == 1:
        #             path = list(self.unique_ip)[0]
        #         else:
        #             return
                    
        #     else:
        #         return
            
        # socket create successfully
        if fd != '-1':
            # AF_UNIX socket
            if self.file_table.get(fd) == "NOT_NET":
                return
            
            # if malware use UDP, we need to map fd to ip addr through sendto
            if syscall['name'] == 'sendto' and not self.file_table.get(fd):
                if 'addr' in info:
                    path = info['addr']

                elif len(self.unique_ip) == 1:
                    path = list(self.unique_ip)[0]

                self.file_table.update(fd, path)
                self.unique_ip.add(path)       
                
            path = self.file_table.get(fd)
            # created in parent process, find it
            if not path:
                path = self._find_path_by_fd(fd, syscall['time'])
        
            
        # file node not exist
        if path not in self._file_node_map:
            child_node = Node(path, typ='n')
            self._file_node_map[path] = child_node
            self.graph[child_node] = []
            self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))

        # file node exist
        else:
            child_node = self._file_node_map[path]
            # if curent node has connected to file node open(), remove open edge
            replace_edge_name = ['connect', 'getsockopt']
            for exist_edge_name in replace_edge_name:
                edge = (self.current_node, child_node, exist_edge_name)
                if edge in self.edges:
                    self._rm_connect(edge)

            # if has conected with send(), don't re-connect current node and child node
            if not self.isDuplicate :
                if (self.current_node, child_node, edge_name) not in self.edges:
                    self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
            else:
                self._connect_node(self.current_node, child_node, Edge(edge_name, syscall['time']))
                
                

                
    def _ioctl(self, syscall):
        info = sys_parser.parse(syscall)
        if info['req'] != 'SIOCGIFADDR':
            return
        
        edge_name = 'read'
        path = info['dev']
        
        self._connect_proc_file(path, edge_name, syscall['time'], from_proc = False, node_typ = 'n')
        
    
    def _rm(self, syscall):
        info = sys_parser.parse(syscall)
        path = info['path']
        
        edge_name = 'rm'
        self._connect_proc_file(path, edge_name, syscall['time'])
        
    def _mkdir(self, syscall):
        info = sys_parser.parse(syscall)
        path = info['path']
        
        edge_name = 'mkdir'
        self._connect_proc_file(path, edge_name, syscall['time'])
        
    def _link(self, syscall):
        info = sys_parser.parse(syscall)
        t = info['target']
        l = info['link']
        
        self._connect_proc_file(t, 'read', syscall['time'], from_proc = False)
        self._connect_proc_file(l, 'write', syscall['time'])
        
    
    def _kill(self, syscall):
        info = sys_parser.parse(syscall)
        pid = info['pid']
        
        self._connect_proc_file(pid, 'kill', syscall['time'], node_typ = 'p')
        
    def _ptrace(self, syscall):
        info = sys_parser.parse(syscall)
        req = info['req'].lower()
        if 'pid' in info:
            self._connect_proc_file(info['pid'], req, syscall['time'], node_typ = 'p')
        else:
            self._connect_proc_file(self.current_node.name, req, syscall['time'], node_typ = 'p')
        
    def _syscall_handler(self, syscall):
        name = syscall['name']
        
        # child process creation
        if name in process_creation:
            self._syscall_handler_table['process'](syscall)
            self.trigger_lines += 1
        
        # file open
        elif name in open_file:
            self._syscall_handler_table['open'](syscall)
            self.trigger_lines += 1
            
        elif name in write_file:
            self._syscall_handler_table['write'](syscall)
            self.trigger_lines += 1
        
        elif name in info:
            self._syscall_handler_table['info'](syscall)
            self.trigger_lines += 1
            
        elif name in send:
            self._syscall_handler_table['send'](syscall)
            self.trigger_lines += 1
        
        elif name in recv:
            self._syscall_handler_table['recv'](syscall)
            self.trigger_lines += 1
            
        elif name in remove:
            self._syscall_handler_table['rm'](syscall)
            self.trigger_lines += 1
            
        elif name in link:
            self._syscall_handler_table['link'](syscall)
            self.trigger_lines += 1
        
        elif name in self._syscall_handler_table:
            self._syscall_handler_table[name](syscall)
            self.trigger_lines += 1

        elif name in self.added_handler_table:
            self._added_handler(syscall)
        #     self.added_handler_table[name](syscall)
        #     self.trigger_lines += 1
    
    # main method to create attck graph
    def create(self):
        total_line = 0
        for idx, trace in enumerate(self.traces):
            #print(trace)
            proc_id = re.search(RE_PID, trace).group(1)
            self.proc_id = proc_id
            # init root node
            if idx == 0:
                self.proc_node_map[proc_id] = Node('malware')
            
            # current process node for this trace
            self.current_node = self.proc_node_map[proc_id]
            self.graph[self.current_node] = []
            self._inside_execve = False
            self.exec = False
            
            first_line = True
            
            # counter = 0
            with open(trace, 'r') as f:
                for line in f:
                    total_line += 1
                    if idx == 0 and first_line:
                        first_line = False
                        continue
                    line = line.strip()
                    call = parse_line(line)
                    
                    if not call:
                        continue

                    if call['type'] == 'sys':
                        # sys_ignore = ['open', 'openat', 'read']
#                         sys_ignore = ['read']
                        # if(call['name'] == 'wait4'):
                        #     print("Call:", call)
                        #     print("Args:", call['args'], type(call['args']))
                        #     print("Args split:", call['args'].split(",") )
                        #     print("Parse :", sys_parser.parse(call))
                        #     break
                        
                        
                        self._syscall_handler(call)
                         #myprint(call, 'socket')
            

        self.traces_lines = total_line
        # print("Total trace lines:", self.traces_lines)

    def sort_edges(self, edges, rm_nodes):
        if len(rm_nodes) != 0:
            edges = [edge for pair, edge in edges.items() if pair[0] not in rm_nodes and pair[1] not in rm_nodes]
        else:
            edges = list(edges.values())
            
        edges.sort(key = lambda edge: datetime.strptime(edge.timestamp, STRACE_TIME_FORMAT))
        self._edges_sort_map = {}
        
        for idx, edge in enumerate(edges):
            self._edges_sort_map[edge] = str(idx + 1)
            
    def node_reduction(self, depth):
        file_counter = {}
        ip_counter = {"External_IP": 0, "Local_IP": 0}
        
        file_set = set()
        ip_set = set()
        
        pub_ip_node = Node('External_IP', typ = 'n')
        pub_ip_node.color = COMMAND_CONTROL_COLOR
        pub_ip_node.tactic ='Command & Control', 
        pub_ip_node.technique = "Command & Control"
        pub_ip_node.id = "C&C"
        
        pri_ip_node = Node('Local_IP', typ = 'n')
    
        reduced_file_map = {}
        nodes = set()
        
        # create another edges for visualization
        node_pairs = []
        edge_vals = []
        for pair, edge in self.edges.items():
            node_pairs.append(pair)
            edge_vals.append(edge)
            
        edges = {}
        for pair_idx, pair in enumerate(node_pairs):
            new_pair = list(pair)
            for idx, node in enumerate(new_pair[:2]):
                # if node is a file or ip addr
                if node.type in ['f', 'n']:
                    if node.name.startswith('8.8.8.8'):
                        nodes.add(node)
                        continue
                        
                    if re.search(ENDPOINT_REGEX, node.name):
                        if re.search(PRIVATE_IP_REGEX, node.name):
                            if node.name not in ip_set:
                                ip_counter['Local_IP'] += 1
                                ip_set.add(node.name)
                                
                            new_pair[idx] = pri_ip_node
                            nodes.add(pri_ip_node)
                        else:
                            if node.name not in ip_set:
                                ip_counter['External_IP'] += 1
                                ip_set.add(node.name)
                                
                            new_pair[idx] = pub_ip_node
                            nodes.add(pub_ip_node)

                    # file path
                    else:
                        paths = node.name.split('/')
                        
                        if len(paths) <= depth:
                            name = "/".join(paths)
                        else:
                            name = "/".join(paths[:depth + 1])
                            

                        if name not in reduced_file_map:
                            if name == "NO_SOCKET" or name[0:3] == "NIC":
                                file_node = Node(name, typ = 'n')
                            else:
                                file_node = Node(name, typ = 'f')
                            file_node.tactic = node.tactic
                            file_node.technique = node.technique
                            file_node.color = node.color
                            file_node.id = node.id
                            
                            reduced_file_map[name] = file_node
                            file_counter[name] = 1
                            file_set.add(node.name)
                            
                        else:
                            file_node = reduced_file_map[name]
                            
                            if node.name not in file_set:
                                file_set.add(node.name)
                                file_counter[name] += 1
                            
                        nodes.add(file_node)
                        new_pair[idx] = file_node
                        
                    
                # if node is process or command       
                else:
                    nodes.add(node)
                    
            # save new pair
            new_pair = tuple(new_pair)
            edges[new_pair] = edge_vals[pair_idx]
            
        # label count number on reduction node
        for node in nodes:
            if node.type == 'n' and node.name in ip_counter:
                count = ip_counter[node.name]
                if count > 1:
                    node.name = node.name + f" ({count})"
            elif node.type == 'f' and node.name in file_counter:
                count = file_counter[node.name]
                if count > 1:
                    node.name = node.name + f" ({count})"
                
            
        return nodes, edges
    
    def is_process_node(self, node):
        return node.type == 'p' and re.search('^[0-9]*$', node.name)
                                  
    def draw(self, showall = True, depth = 3):
        # Create Digraph object
        display_threshold = 30
        # twopi
        dot = Digraph(engine='dot', format='pdf')
        #dot.attr(size='35,50')
        #dot.graph_attr['overlap'] =  "false"
        #dot.graph_attr['splines'] = "true"
        
        node_table = {}
        nodes = list(self.graph.keys())
        edges = dict(self.edges)
        
        self.all_nodes_num = len(nodes)
        print('Node before reduction: ', self.all_nodes_num)
        self.color_nodes_num = 0
        
        
        # nodes to be removed
        rm_nodes = set()
        
        # remove node that have only on edge clone from parent
        if not showall:
            # node have no child node
            for node in nodes:
                # if a node have no child node
                if self.is_process_node(node):
                    if len(self.graph[node]) == 0:
                        rm_nodes.add(node)
                    
            for node in nodes:
                if len(self.graph[node]) == 1 and self.graph[node][0] in rm_nodes:
                    rm_nodes.add(node)

            # node in rm_nodes that have only one parent node which its edge in process_creation
            for pair, edge in edges.items():
                from_node, to_node, _ = pair
                if to_node in rm_nodes and edge.name not in process_creation:
                    rm_nodes.remove(to_node)
            
            for node in rm_nodes:
                print(f'Remove Node: {node.name}')
        
        # print(self.all_nodes_num - len(rm_nodes))
        if (self.all_nodes_num - len(rm_nodes)) > display_threshold and not showall:
            nodes, edges = self.node_reduction(depth)
            
        
        # visulization node counting
        self.viz_nodes_num = len(nodes) - len(rm_nodes)
        print('Nodes after reduction: ', self.viz_nodes_num)
        # sort edge
        self.sort_edges(edges, rm_nodes)
        
        # draw node
        for idx, node in enumerate(nodes):
            idx = str(idx)
            
            if not showall and node in rm_nodes:
                continue
            
            
            node_name = node.name
            if node.technique:
                 node_name += f'\n({node.id})'
            
            if node.color:
                self.color_nodes_num += 1

            if node.type == 'n':
                dot.node(idx, node_name, shape='diamond', color = node.color, style='filled')
            elif node.type in ['f', 'c']:
                dot.node(idx, node_name, shape='rectangle', color = node.color, style='filled')
            # elif node.type == 'n':
            #     dot.node(idx, node_name, shape='diamond', color = node.color, style='filled')
            elif node.type == 'm_addr':
                dot.node(idx, node_name, shape='pentagon', color = node.color, style='filled')
            elif node.type == 'pipe':
                dot.node(idx, node_name, shape='cylinder', color = node.color, style='filled')
            elif node.type == 'else':
                dot.node(idx, node_name, shape='tab', color = node.color, style='filled')
            else:
                dot.node(idx, node_name, color = node.color, style='filled')
                
            node_table[node] = idx
        
        self.node_seq = []
        
        # draw edge
        for pair, edge in edges.items():
            from_node, to_node, _ = pair
            
            if not showall and (from_node in rm_nodes or to_node in rm_nodes):
                continue
            
            self.node_seq.append([self._edges_sort_map[edge], from_node, to_node, edge])
            
            from_node = node_table[from_node]
            to_node = node_table[to_node]
            
            edge_name = self._edges_sort_map[edge] + '. ' + edge.name
            dot.edge(from_node, to_node, edge_name, color = edge.color)
        
        # sort node sequence
        self.node_seq = sorted(self.node_seq, key=lambda x: int(x[0]))
        
        return dot

In [111]:
re.findall( r"\d+","[{fd=3', ' events=POLLIN}, {fd=4', ' events=POLLIN}]")

['3', '4']

In [75]:
path = './CCOE/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin'

graph = AttackGraph(path)
graph.create()

In [71]:
len(graph.step_list)

8007

In [67]:
len(graph.step_list)

8007

In [76]:
import pickle

with open('Dofloo_Dofloo_V2_ALL_Memory_sink.pkl', 'wb') as outp:
    pickle.dump(graph, outp, pickle.HIGHEST_PROTOCOL)

In [88]:
print(graph.file_table)

-----------
-------------



In [63]:
# Create TT Graph based on AS Graph
mapper = TTPLabeler(graph)
mapper.fit()
g = graph.draw(False)
mal_name = path.split("/")[2]
# print(mal_name)
g.render("./output-img/" + "dofloo/1115/" + mal_name + "_test", format="svg")
# g.render("./output-img/" + mal_name + "(add_sys)", format="png")

Node before reduction:  92
Nodes after reduction:  68


'output-img/dofloo/1115/Dofloo_test.svg'

In [17]:
# Create TT Graph based on AS Graph
mapper = TTPLabeler(graph)
mapper.fit()
g = graph.draw(False)
mal_name = path.split("/")[2]
# print(mal_name)
# g.render("./output-img/" + mal_name, format="svg")
g.render("./output-img/" + mal_name + "_test", format="png")

Node before reduction:  30
Remove Node: 1529
Remove Node: 1527
Remove Node: 1528
Nodes after reduction:  27


'output-img/Dofloo_test.png'

In [53]:
# wait4, mmap2
# _newselect have fd  sets
# lseek fd = 0
s = "2, NULL, [1], NULL, {tv_sec=5, tv_usec=0}"
print(s.split(",")[0])
print(s.split(",")[3].strip(" ") == "NULL", s.split(",")[1])

True  NULL


# MITRE Mapping Rules

In [16]:
DISCOVERY_COLOR = '#FDE74C'
PERSISTENCE_COLOR = '#9BC53D'
EXECUTION_COLOR = '#26C485'
COMMAND_CONTROL_COLOR = '#CC5A71'

DISCOVERY = [{
    "tactic": "Discovery",
    "technique": "System Information Discovery",
    "id": "T1082",
    "file_edges": ['open', 'read'],
    "command_edges": ['exec'],
    "commands": ['lspci', 'dmidecode', 'lscpu', 'lsmod', 'hostname'],
    "files": ['/etc/lsb-release', '/etc/redhat-release', '/etc/issue', 'sysinfo', 'uname', 
              '/proc/cpuinfo', '/proc/meminfo', '/proc/stat', '/proc/version', '/proc/sys/kernel/version', 
              '/proc/sys/kernel/ngroups_max', '/sys/'],
    "color": DISCOVERY_COLOR
    
}, {
    "tactic": "Discovery",
    "technique": "System Network Connection Discovery",
    "id": 'T1049',
    "file_edges": ['open', 'read', 'connect'],
    "command_edges": ['exec'],
    "commands": ['netstat', 'lsof', 'who', 'w'],
    "files": ['/proc/net/dev', '/proc/net/tcp', '/proc/net/route', '8\.8\.8\.8'],
    "color": DISCOVERY_COLOR
}, {
    "tactic": "Discovery",
    "technique": "System Network Configuration Discovery",
    "id": "T1016",
    "file_edges": ['open', 'read'],
    "command_edges": ['exec'],
    "commands": ['arp', 'ifconfig', 'ip'],
    "files": ['/proc/net/arp', '/proc/net/unix', '/etc/nsswitch.conf', '/etc/resolv.conf',
              '/etc/host', 'eth'],
    "color": DISCOVERY_COLOR
}]

PERSISTENCE = [{
    "tactic": "Persistence",
    "technique": "Local Job Scheduling",
    "id": "T1168",
    "file_edges": ['open', 'write'],
    "command_edges": ['exec', 'clone', 'fork', 'vfork'],
    "commands": ['crontab'],
    "files": ['/etc/rc', '/etc/init', '/etc/cron', '/var/spool/cron'],
    'color': PERSISTENCE_COLOR
}]

EXECUTION = [{
    "tactic": "Execution",
    "technique": "Command-Line Interface",
    "id": 'T1059',
    "file_edges": [],
    "command_edges": ['clone', 'exec', 'fork', 'vfork'],
    "commands": ['sh', 'bash', 'dash', 'rbash'],
    "files": [],
    "color": EXECUTION_COLOR
}]

COMMAND_CONTROL = [{
    "tactic": "Command & Control",
    "technique": "Command & Control",
    "id": "C&C",
    "file_edges": [],
    "command_edges": ['exec', 'clone', 'fork', 'vfork'],
    "commands": ['nc', 'wget', 'curl'],
    "files": [],
    "color": COMMAND_CONTROL_COLOR
}]

# Tactic & Technique Graph Generation

In [17]:
# Generation of TT Graph
class TTPLabeler:
    def __init__(self, attack_graph):
        self.graph = attack_graph
        
    def set_technique(self, node, edge, rule):
        #print(edge.name, node.name)
        node.technique = rule['technique']
        node.tactic = rule['tactic']
        node.color = rule['color']
        node.id = rule['id']
        
        edge.technique = rule['technique']
        edge.tactic = rule['tactic']
        edge.color = rule['color']
    
    def mapper(self, rules):
        for rule in rules:            
            # map files
            if self.edge.name in rule['file_edges']:
                for node in self.nodes:
                    if node.type in ['f', 'n', 'c']:                                
                        for file in rule['files']:
                            if re.search(file, node.name):
                                self.set_technique(node, self.edge, rule)
                                return
            
            
            # map commands
            if self.edge.name in rule['command_edges']:
                for node in self.nodes:
                    if node.type == 'p':
                        for command in rule['commands']:
                            if re.search(f'^{command}$', node.name):
                                self.set_technique(node, self.edge, rule)
                                return
                            
    
    def execution_mapper(self):
        self.mapper(EXECUTION)
        
    def discovery_mapper(self):
        self.mapper(DISCOVERY)
    
    def persistence_mapper(self):
        self.mapper(PERSISTENCE)
        
    def command_control_mapper(self):
        self.mapper(COMMAND_CONTROL)
        
        # label external ip connection
        for node in self.nodes:
            if node.type != 'n':
                continue
            if node.name.startswith('8.8.8.8'):
                continue
                
            if self.edge.name not in ['send', 'recv', 'connect', 'read', 'write']:
                continue
                
            if re.search(ENDPOINT_REGEX, node.name) and not re.search(PRIVATE_IP_REGEX, node.name):
                self.set_technique(node, self.edge, COMMAND_CONTROL[0])
    
    def fit(self):
        edges = self.graph.edges
        for nodes, edge in edges.items():
            #print(nodes[0].name, nodes[1].name)
            self.nodes = nodes[:2]
            self.edge = edge
            
            # map each tactic
            self.discovery_mapper()
            self.persistence_mapper()
            self.execution_mapper()
            self.command_control_mapper()

# Experient

path: 設為動態側錄系統產生的惡意程式 traces 的路徑

In [20]:
# path = './root_real_network/Backdoor/Dofloo/fb3cf2464f43906cb96523752d74e5ad8e4eccf8ab74890e51fe24266921ef1c'
path = './XORDDOS/ddos/0aefb67c01a24d05351b093455203fa2'

In [12]:
# Statistic for family
import os
import re

p_list = ["rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "capget", "capset", "nanosleep", "alarm" ,"setitimer" ,"getitimer", "sched_yield", "setns", "mprotect", "brk", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "mremap", "mincore", "pause", "clone", "fork", "vfork", "execve", "exit", "wait4", "kill", "ptrace", "uselib", "personality", "sched_setparam", "sched_getparam", "sched_setscheduler", "sched_getscheduler", "sched_rr_get_interval", "mlock", "munlock", "mlockall", "munlockall", "vhangup", "prctl", "arch_prctl", "create_module", "init_module", "query_module", "delete_module", "tkill", "futex", "sched_setaffinity", "sched_getaffinity", "set_thread_area", "get_thread_area", "set_tid_address", "restart_syscall", "timer_create", "timer_settime", "timer_gettime", "timer_getoverrun", "timer_delete", "exit_group", "tgkill", "mbind", "set_mempolicy", "get_mempolicy", "waitid", "migrate_pages", "unshare", "set_robust_list", "get_robust_list", "ioprio_set", "ioprio_get", "move_pages", "rt_tgsigqueueinfo", "process_vm_readv", "process_vm_writev", "kcmp", "finit_module", "sched_setattr", "sched_getattr", "seccomp", "getrandom", "bpf", "execveat", "membarrier", "mlock2", "pkey_mprotect", "pkey_alloc", "pkey_free"]
p_list_arm = ["cacheflush", "breakpoint", "cmpxchg", "vm86", "bdflush", "sigprocmask", "sigreturn", "syscall", "idle", "sgetmask", "ssetmask", "sigsuspend", "sigpending", "sigaction", "lock", "signal", "nice", "break", "waitpid", "rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "capget", "capset", "nanosleep", "alarm" ,"setitimer" ,"getitimer", "sched_yield", "setns", "mprotect", "brk", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "mremap", "mincore", "pause", "clone", "fork", "vfork", "execve", "exit", "wait4", "kill", "ptrace", "uselib", "personality", "sched_setparam", "sched_getparam", "sched_setscheduler", "sched_getscheduler", "sched_rr_get_interval", "mlock", "munlock", "mlockall", "munlockall", "vhangup", "prctl", "create_module", "init_module", "query_module", "delete_module", "tkill", "futex", "sched_setaffinity", "sched_getaffinity", "set_tid_address", "restart_syscall", "timer_create", "timer_settime", "timer_gettime", "timer_getoverrun", "timer_delete", "exit_group", "tgkill", "mbind", "set_mempolicy", "get_mempolicy", "waitid", "migrate_pages", "unshare", "set_robust_list", "get_robust_list", "ioprio_set", "ioprio_get", "move_pages", "rt_tgsigqueueinfo", "process_vm_readv", "process_vm_writev", "kcmp", "finit_module", "sched_setattr", "sched_getattr", "seccomp", "getrandom", "bpf", "execveat", "membarrier", "mlock2", "pkey_mprotect", "pkey_alloc", "pkey_free"]
p_list = list(set(p_list+p_list_arm))

f_list = ["munmap", "mmap", "putpmsg", "getpmsg", "utime", "read", "write", "open", "close", "stat", "fstat", "lstat", "poll", "lseek", "pread64", "pwrite64", "readv", "writev", "access", "select", "msync", "dup", "dup2", "sendfile", "fcntl", "flock", "fsync", "fdatasync", "truncate", "ftruncate", "getdents", "getcwd", "chdir", "fchdir", "rename", "mkdir", "rmdir", "creat", "link", "unlink", "symlink", "readlink", "chmod", "fchmod", "chown", "fchown", "lchown", "umask", "mknod", "statfs", "fstatfs", "sysfs", "modify_ldt", "pivot_root", "sync", "acct", "mount", "umount2", "readahead", "setxattr", "lsetxattr", "fsetxattr", "getxattr", "lgetxattr", "fgetxattr", "listxattr", "llistxattr", "flistxattr", "removexattr", "lremovexattr", "fremovexattr", "lookup_dcookie", "epoll_create", "epoll_ctl_old", "epoll_wait_old", "remap_file_pages", "getdents64", "fadvise64", "epoll_wait", "epoll_ctl", "utimes", "inotify_init", "inotify_add_watch", "inotify_rm_watch", "openat", "mkdirat", "mknodat", "fchownat", "futimesat", "newfstatat", "unlinkat", "renameat", "linkat", "symlinkat", "readlinkat", "fchmodat", "faccessat", "pselect6", "ppoll", "splice", "tee", "sync_file_range", "vmsplice", "utimensat", "epoll_pwait", "signalfd", "timerfd_create", "eventfd",  "fallocate", "timerfd_settime", "timerfd_gettime", "signalfd4", "eventfd2", "epoll_create1", "dup3", "inotify_init1", "preadv", "pwritev", "fanotify_init", "fanotify_mark", "name_to_handle_at", "open_by_handle_at", "syncfs", "renameat2", "memfd_create", "userfaultfd", "copy_file_range", "preadv2", "pwritev2", "statx", "chroot"]
f_list_arm = ["sync_file_range2", "fstatat64", "fadvise64_64", "fstatfs64", "statfs64", "sendfile64", "fcntl64", "chown32", "fchown32", 'stat64', 'lstat64', 'fstat64', 'lchown32', "ftruncate64", "truncate64", "mmap2", "_newselect", "_llseek", "readdir", "oldlstat", "oldfstat", "umount", "oldstat", "munmap", "mmap", "putpmsg", "getpmsg", "utime", "read", "write", "open", "close", "stat", "fstat", "lstat", "poll", "lseek", "pread64", "pwrite64", "readv", "writev", "access", "select", "msync", "dup", "dup2", "sendfile", "fcntl", "flock", "fsync", "fdatasync", "truncate", "ftruncate", "getdents", "getcwd", "chdir", "fchdir", "rename", "mkdir", "rmdir", "creat", "link", "unlink", "symlink", "readlink", "chmod", "fchmod", "chown", "fchown", "lchown", "umask", "mknod", "statfs", "fstatfs", "sysfs", "modify_ldt", "pivot_root", "sync", "acct", "mount", "umount2", "readahead", "setxattr", "lsetxattr", "fsetxattr", "getxattr", "lgetxattr", "fgetxattr", "listxattr", "llistxattr", "flistxattr", "removexattr", "lremovexattr", "fremovexattr", "lookup_dcookie", "epoll_create", "remap_file_pages", "getdents64", "epoll_wait", "epoll_ctl", "utimes", "inotify_init", "inotify_add_watch", "inotify_rm_watch", "openat", "mkdirat", "mknodat", "fchownat", "futimesat", "unlinkat", "renameat", "linkat", "symlinkat", "readlinkat", "fchmodat", "faccessat", "pselect6", "ppoll", "splice", "tee", "vmsplice", "utimensat", "epoll_pwait", "signalfd", "timerfd_create", "eventfd",  "fallocate", "timerfd_settime", "timerfd_gettime", "signalfd4", "eventfd2", "epoll_create1", "dup3", "inotify_init1", "preadv", "pwritev", "fanotify_init", "fanotify_mark", "name_to_handle_at", "open_by_handle_at", "syncfs", "renameat2", "memfd_create", "userfaultfd", "copy_file_range", "preadv2", "pwritev2", "statx", "chroot"]
f_list = list(set(f_list+f_list_arm))

d_list = ["ioctl", "swapon", "swapoff", "iopl", "ioperm", "quotactl", "io_setup", "io_destroy", "io_getevents", "io_submit", "io_cancel", "io_pgetevents"]
d_list_arm = ['pciconfig_read', 'pciconfig_write', "pciconfig_iobase", "ioctl", "swapon", "swapoff", "iopl", "ioperm", "quotactl", "io_setup", "io_destroy", "io_getevents", "io_submit", "io_cancel", "io_pgetevents"]
d_list = list(set(d_list+d_list_arm))

info_list = ["nfsservctl", "madvise", "reboot", "kexec_load", "kexec_file_load", "add_key", "request_key", "keyctl", "perf_event_open", "getpid", "uname", "semget", "semop", "semctl", "msgget", "msgctl", "gettimeofday", "getrlimit", "getrusage", "sysinfo", "times", "getuid", "syslog", "getgid", "setuid", "setgid", "geteuid", "getegid", "setpgid", "getppid", "getpgrp", "setsid", "setreuid", "setregid", "getgroups", "setgroups", "setresuid", "getresuid", "setresgid", "getresgid", "getpgid", "setfsuid", "setfsgid", "getsid", "ustat", "getpriority", "setpriority", "sched_get_priority_max", "sched_get_priority_min", "_sysctl", "adjtimex", "setrlimit", "settimeofday", "sethostname", "setdomainname", "get_kernel_syms", "gettid", "time", "semtimedop", "clock_settime", "clock_gettime", "clock_getres", "clock_nanosleep", "prlimit64", "clock_adjtime", "getcpu", "shmget", "shmat", "shmctl", "shmdt"]
info_list_arm = ['setuid32', 'setgid32', 'setfsuid32', 'setfsgid32', 'setresuid32', 'getresuid32', 'setresgid32', 'getresgid32', 'setregid32', 'getgroups32', 'setgroups32', 'getuid32', 'getgid32', 'geteuid32', 'getegid32', 'setreuid32', "ugetrlimit", "ipc", "olduname", "profil", "oldolduname", "ulimit", "ftime", "stty", "stime", "nfsservctl", "madvise", "reboot", "kexec_load", "kexec_file_load", "add_key", "request_key", "keyctl", "perf_event_open", "getpid", "uname", "semget", "semop", "semctl", "msgget", "msgctl", "gettimeofday", "getrlimit", "getrusage", "sysinfo", "times", "getuid", "syslog", "getgid", "setuid", "setgid", "geteuid", "getegid", "setpgid", "getppid", "getpgrp", "setsid", "setreuid", "setregid", "getgroups", "setgroups", "setresuid", "getresuid", "setresgid", "getresgid", "getpgid", "setfsuid", "setfsgid", "getsid", "ustat", "getpriority", "setpriority", "sched_get_priority_max", "sched_get_priority_min", "_sysctl", "adjtimex", "setrlimit", "settimeofday", "sethostname", "setdomainname", "get_kernel_syms", "gettid", "time", "semtimedop", "clock_settime", "clock_gettime", "clock_getres", "clock_nanosleep", "prlimit64", "clock_adjtime", "getcpu", "shmget", "shmat", "shmctl", "shmdt"]
info_list = list(set(info_list+info_list_arm))

commu_list = ["mq_open", "mq_unlink", "mq_timedsend", "mq_timedreceive", "mq_notify", "mq_getsetattr", "pipe", "socket", "connect", "accept", "sendto", "recvfrom", "sendmsg", "recvmsg", "shutdown", "bind", "listen", "getsockname", "getpeername", "socketpair", "setsockopt", "getsockopt", "msgsnd", "msgrcv", "accept4", "pipe2", "recvmmsg", "sendmmsg"]
commu_list_arm = ["mq_open", "mq_unlink", "mq_timedsend", "mq_timedreceive", "mq_notify", "mq_getsetattr", 'send', 'recv', "socketcall", "pipe", "socket", "connect", "accept", "sendto", "recvfrom", "sendmsg", "recvmsg", "shutdown", "bind", "listen", "getsockname", "getpeername", "socketpair", "setsockopt", "getsockopt", "msgsnd", "msgrcv", "accept4", "pipe2", "recvmmsg", "sendmmsg"]
commu_list = list(set(commu_list+commu_list_arm))

sys_val_dict = {"Porcess": p_list, "File": f_list, "Device": d_list, "Infomation": info_list, "Communication": commu_list}
# sys_val_dict_arm = {"Porcess": p_list_arm, "File": f_list_arm, "Device": d_list_arm, "Infomation": info_list_arm, "Communication": commu_list_arm}

def parse_line(log):
    signal = re.search('--- (SIG.*?) {(.*)}', log)
    term = re.search('[0-9]* \+\+\+ (.*) \+\+\+', log)
    
    info = {}
    
    if signal:
        info['type'] = 'sig'
        info['name'] = signal.group(1)
        info['args'] = signal.group(2)
    elif term:
        info['type'] = 'term'
        info['status'] = term.group(1)
    else: 
        info['type'] = 'sys'
        g = re.search(' (.*?)\((.*)\).*= (.*)', log)
        
        # uncompleted function caused by terminating execution
        if not g:
            return None
        
        info['name'] = g.group(1)
        info['args'] = g.group(2)
        info['return'] = g.group(3)
        info['success'] = True
        info['return_msg'] = None
        
        
        # return status: True for success, False for error (startswith E or ?)
        ret = info['return'].split(' ')
        if len(ret) > 1:
            info['return'] = ret[0]
            info['return_msg'] = ' '.join(ret[1:])
            
            # inspect first char of return description ()
            if ret[1].startswith('E'):
                info['success'] = False
                info['return_msg'] = ' '.join(ret[2:])
                
        if info['return'] == '?':
            info['success'] = False
    
    time = re.search(RE_TIME, log).group(1)
    info['time'] = time
    
    return info

def get_sys_dict(path):
    sys_dict = {}
    traces = sorted(glob.glob(path + '/strace*'))
    # print(traces)

    total_line = 0
    for idx, trace in enumerate(traces):
        
        with open(trace, 'r') as f:
            for line in f:
                # print("Open")
                line = line.strip()
                call = parse_line(line)
                
                if not call:
                    continue

                if call['type'] == 'sys':
                    total_line += 1
                    
                    sys_name = call['name']
                    # print(sys_name)
                    if(sys_name in sys_dict):
                        sys_dict[sys_name] += 1
                    else:
                        sys_dict[sys_name] = 1
    return sys_dict
                    

def get_sys_arrange(sys_dict):

    sys_arrange = {"Porcess": 0, "File": 0, "Device": 0, "Infomation": 0, "Communication": 0}
    for sys_name in sys_dict:

        for sys_type in sys_val_dict:
            sys_list = sys_val_dict[sys_type]

            if(sys_name in sys_list):
                sys_arrange[sys_type] += sys_dict[sys_name]
                break

            if(sys_type == "Communication"):
                print(sys_name)
                print("Error!!!")

    return sys_arrange



# dir = "./CCOE/Dofloo/"
# mal_hash = os.listdir(dir)


# for hash in mal_hash:
#     path = dir + hash
#     print("Family:", path.split("/")[2], "Hash:", path.split("/")[3])
#     sys_dict = get_sys_dict(path)
#     sys_dict = {k: v for k, v in sorted(sys_dict.items(), key=lambda item: item[1])}
#     sys_arrange = get_sys_arrange(sys_dict)

#     total_lines =  0
#     for key in sys_arrange:
#         total_lines += sys_arrange[key]
#     print("Trace lines:", total_lines)
#     print(sys_arrange)
#     print(sys_dict)
#     print("Num of syscall type:", len(sys_dict))

#     if(len(list(sys_dict.keys()))>= 1):
#         sys_name = list(sys_dict.keys())[-1]
#         invoke_times = sys_dict[list(sys_dict.keys())[-1]]
#     else:
#         sys_name = ""
#         invoke_times = 0

#     print("Max syscall:[" +  sys_name + "]", "Invoke times:", invoke_times)

#     for type in sys_val_dict:
#         sys_list = sys_val_dict[type]
#         if(sys_name in sys_list):
#             print("Syscall type:", [type])
#             break
    
#     print("-"*100)


# print("Num of syscall type:", len(sys_dict))

# if(len(list(sys_dict.keys()))>= 1):
#     sys_name = list(sys_dict.keys())[-1]
#     invoke_times = sys_dict[list(sys_dict.keys())[-1]]
# else:
#     sys_name = ""
#     invoke_times = 0

# print("Max syscall:[" +  sys_name + "]", "Invoke times:", invoke_times)

# for type in sys_val_dict:
#     sys_list = sys_val_dict[type]
#     if(sys_name in sys_list):
#         print("Syscall type:", [type])
#         break




In [15]:
# Step 2 : 去除 signal、exit()

path = "./CCOE/Xorddos/0aefb67c01a24d05351b093455203fa2.bin"

print("Family:", path.split("/")[2], "Hash:", path.split("/")[3])
sys_dict = get_sys_dict(path)
sys_dict = {k: v for k, v in sorted(sys_dict.items(), key=lambda item: item[1])}
sys_arrange = get_sys_arrange(sys_dict)

total_lines =  0
for key in sys_arrange:
    total_lines += sys_arrange[key]
print("Trace lines:", total_lines)
print(sys_arrange)
print(sys_dict)

p_sig = ["rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "personality", "rt_tgsigqueueinfo", ]

remained_lines = 0
cut_sys_num = 0
for key in sys_dict:
    if(key not in p_sig and key != "exit"):
        remained_lines += sys_dict[key]
    else:
        print("Cut:", key)
        cut_sys_num += 1
print("Remained syscall type:", len(sys_dict) - cut_sys_num)
print("Remained lines:", remained_lines)
print("Cut lines:", total_lines - remained_lines)

Family: Xorddos Hash: 0aefb67c01a24d05351b093455203fa2.bin
Trace lines: 448425
{'Porcess': 54453, 'File': 387601, 'Device': 10, 'Infomation': 4104, 'Communication': 2257}
{'shmget': 1, 'shmat': 1, 'shmdt': 1, 'pipe': 1, 'getppid': 1, 'rt_sigreturn': 1, 'fchown': 1, 'fchmod': 1, 'rename': 1, 'getsockname': 1, 'wait4': 2, 'umask': 2, 'setsockopt': 2, 'sendmsg': 2, 'prlimit64': 3, 'getpid': 3, 'newfstatat': 3, 'arch_prctl': 4, 'munmap': 4, 'getuid': 4, 'getgid': 4, 'getegid': 4, 'statfs': 4, 'fcntl': 5, 'lstat': 5, 'geteuid': 8, 'symlink': 10, 'ioctl': 10, 'getdents': 20, 'connect': 44, 'access': 46, 'getsockopt': 50, 'sendto': 51, 'fstat': 67, 'openat': 81, 'stat': 81, 'mprotect': 85, 'fcntl64': 86, '_newselect': 93, 'socket': 95, 'mmap': 108, 'ppoll': 126, 'set_thread_area': 292, 'ugetrlimit': 292, 'setsid': 292, 'fstat64': 292, 'mmap2': 293, 'futex': 294, 'set_tid_address': 295, 'set_robust_list': 299, 'waitpid': 304, 'execve': 316, 'unlink': 356, 'uname': 584, 'gettimeofday': 608, 'ls

In [16]:
# Step 4
ignore_sys = ["read"]
remained_lines = 0
cut_sys_num = 0
for key in sys_dict:
    if(key not in p_sig and key != "exit" and key not in ignore_sys):
        remained_lines += sys_dict[key]
    else:
        print("Cut:", key)
        cut_sys_num += 1
print("Remained syscall type:", len(sys_dict) - cut_sys_num)
print("Remained lines:", remained_lines)
print("Cut lines:", total_lines - remained_lines)

Cut: rt_sigreturn
Cut: rt_sigaction
Cut: rt_sigprocmask
Cut: read
Remained syscall type: 68
Remained lines: 380427
Cut lines: 67998


In [2]:
# Step 3 : 研究剩下的 System call，並繪製 ASG

import os
import re

p_list = ["rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "capget", "capset", "nanosleep", "alarm" ,"setitimer" ,"getitimer", "sched_yield", "setns", "mprotect", "brk", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "mremap", "mincore", "pause", "clone", "fork", "vfork", "execve", "exit", "wait4", "kill", "ptrace", "uselib", "personality", "sched_setparam", "sched_getparam", "sched_setscheduler", "sched_getscheduler", "sched_rr_get_interval", "mlock", "munlock", "mlockall", "munlockall", "vhangup", "prctl", "arch_prctl", "create_module", "init_module", "query_module", "delete_module", "tkill", "futex", "sched_setaffinity", "sched_getaffinity", "set_thread_area", "get_thread_area", "set_tid_address", "restart_syscall", "timer_create", "timer_settime", "timer_gettime", "timer_getoverrun", "timer_delete", "exit_group", "tgkill", "mbind", "set_mempolicy", "get_mempolicy", "waitid", "migrate_pages", "unshare", "set_robust_list", "get_robust_list", "ioprio_set", "ioprio_get", "move_pages", "rt_tgsigqueueinfo", "process_vm_readv", "process_vm_writev", "kcmp", "finit_module", "sched_setattr", "sched_getattr", "seccomp", "getrandom", "bpf", "execveat", "membarrier", "mlock2", "pkey_mprotect", "pkey_alloc", "pkey_free"]
p_list_arm = ["cacheflush", "breakpoint", "cmpxchg", "vm86", "bdflush", "sigprocmask", "sigreturn", "syscall", "idle", "sgetmask", "ssetmask", "sigsuspend", "sigpending", "sigaction", "lock", "signal", "nice", "break", "waitpid", "rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "capget", "capset", "nanosleep", "alarm" ,"setitimer" ,"getitimer", "sched_yield", "setns", "mprotect", "brk", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "mremap", "mincore", "pause", "clone", "fork", "vfork", "execve", "exit", "wait4", "kill", "ptrace", "uselib", "personality", "sched_setparam", "sched_getparam", "sched_setscheduler", "sched_getscheduler", "sched_rr_get_interval", "mlock", "munlock", "mlockall", "munlockall", "vhangup", "prctl", "create_module", "init_module", "query_module", "delete_module", "tkill", "futex", "sched_setaffinity", "sched_getaffinity", "set_tid_address", "restart_syscall", "timer_create", "timer_settime", "timer_gettime", "timer_getoverrun", "timer_delete", "exit_group", "tgkill", "mbind", "set_mempolicy", "get_mempolicy", "waitid", "migrate_pages", "unshare", "set_robust_list", "get_robust_list", "ioprio_set", "ioprio_get", "move_pages", "rt_tgsigqueueinfo", "process_vm_readv", "process_vm_writev", "kcmp", "finit_module", "sched_setattr", "sched_getattr", "seccomp", "getrandom", "bpf", "execveat", "membarrier", "mlock2", "pkey_mprotect", "pkey_alloc", "pkey_free"]
p_list = list(set(p_list+p_list_arm))
p_sig = ["rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "personality", "rt_tgsigqueueinfo", ]

f_list = ["munmap", "mmap", "putpmsg", "getpmsg", "utime", "read", "write", "open", "close", "stat", "fstat", "lstat", "poll", "lseek", "pread64", "pwrite64", "readv", "writev", "access", "select", "msync", "dup", "dup2", "sendfile", "fcntl", "flock", "fsync", "fdatasync", "truncate", "ftruncate", "getdents", "getcwd", "chdir", "fchdir", "rename", "mkdir", "rmdir", "creat", "link", "unlink", "symlink", "readlink", "chmod", "fchmod", "chown", "fchown", "lchown", "umask", "mknod", "statfs", "fstatfs", "sysfs", "modify_ldt", "pivot_root", "sync", "acct", "mount", "umount2", "readahead", "setxattr", "lsetxattr", "fsetxattr", "getxattr", "lgetxattr", "fgetxattr", "listxattr", "llistxattr", "flistxattr", "removexattr", "lremovexattr", "fremovexattr", "lookup_dcookie", "epoll_create", "epoll_ctl_old", "epoll_wait_old", "remap_file_pages", "getdents64", "fadvise64", "epoll_wait", "epoll_ctl", "utimes", "inotify_init", "inotify_add_watch", "inotify_rm_watch", "openat", "mkdirat", "mknodat", "fchownat", "futimesat", "newfstatat", "unlinkat", "renameat", "linkat", "symlinkat", "readlinkat", "fchmodat", "faccessat", "pselect6", "ppoll", "splice", "tee", "sync_file_range", "vmsplice", "utimensat", "epoll_pwait", "signalfd", "timerfd_create", "eventfd",  "fallocate", "timerfd_settime", "timerfd_gettime", "signalfd4", "eventfd2", "epoll_create1", "dup3", "inotify_init1", "preadv", "pwritev", "fanotify_init", "fanotify_mark", "name_to_handle_at", "open_by_handle_at", "syncfs", "renameat2", "memfd_create", "userfaultfd", "copy_file_range", "preadv2", "pwritev2", "statx", "chroot"]
f_list_arm = ["sync_file_range2", "fstatat64", "fadvise64_64", "fstatfs64", "statfs64", "sendfile64", "fcntl64", "chown32", "fchown32", 'stat64', 'lstat64', 'fstat64', 'lchown32', "ftruncate64", "truncate64", "mmap2", "_newselect", "_llseek", "readdir", "oldlstat", "oldfstat", "umount", "oldstat", "munmap", "mmap", "putpmsg", "getpmsg", "utime", "read", "write", "open", "close", "stat", "fstat", "lstat", "poll", "lseek", "pread64", "pwrite64", "readv", "writev", "access", "select", "msync", "dup", "dup2", "sendfile", "fcntl", "flock", "fsync", "fdatasync", "truncate", "ftruncate", "getdents", "getcwd", "chdir", "fchdir", "rename", "mkdir", "rmdir", "creat", "link", "unlink", "symlink", "readlink", "chmod", "fchmod", "chown", "fchown", "lchown", "umask", "mknod", "statfs", "fstatfs", "sysfs", "modify_ldt", "pivot_root", "sync", "acct", "mount", "umount2", "readahead", "setxattr", "lsetxattr", "fsetxattr", "getxattr", "lgetxattr", "fgetxattr", "listxattr", "llistxattr", "flistxattr", "removexattr", "lremovexattr", "fremovexattr", "lookup_dcookie", "epoll_create", "remap_file_pages", "getdents64", "epoll_wait", "epoll_ctl", "utimes", "inotify_init", "inotify_add_watch", "inotify_rm_watch", "openat", "mkdirat", "mknodat", "fchownat", "futimesat", "unlinkat", "renameat", "linkat", "symlinkat", "readlinkat", "fchmodat", "faccessat", "pselect6", "ppoll", "splice", "tee", "vmsplice", "utimensat", "epoll_pwait", "signalfd", "timerfd_create", "eventfd",  "fallocate", "timerfd_settime", "timerfd_gettime", "signalfd4", "eventfd2", "epoll_create1", "dup3", "inotify_init1", "preadv", "pwritev", "fanotify_init", "fanotify_mark", "name_to_handle_at", "open_by_handle_at", "syncfs", "renameat2", "memfd_create", "userfaultfd", "copy_file_range", "preadv2", "pwritev2", "statx", "chroot"]
f_list = list(set(f_list+f_list_arm))

d_list = ["ioctl", "swapon", "swapoff", "iopl", "ioperm", "quotactl", "io_setup", "io_destroy", "io_getevents", "io_submit", "io_cancel", "io_pgetevents"]
d_list_arm = ['pciconfig_read', 'pciconfig_write', "pciconfig_iobase", "ioctl", "swapon", "swapoff", "iopl", "ioperm", "quotactl", "io_setup", "io_destroy", "io_getevents", "io_submit", "io_cancel", "io_pgetevents"]
d_list = list(set(d_list+d_list_arm))

info_list = ["nfsservctl", "madvise", "reboot", "kexec_load", "kexec_file_load", "add_key", "request_key", "keyctl", "perf_event_open", "getpid", "uname", "semget", "semop", "semctl", "msgget", "msgctl", "gettimeofday", "getrlimit", "getrusage", "sysinfo", "times", "getuid", "syslog", "getgid", "setuid", "setgid", "geteuid", "getegid", "setpgid", "getppid", "getpgrp", "setsid", "setreuid", "setregid", "getgroups", "setgroups", "setresuid", "getresuid", "setresgid", "getresgid", "getpgid", "setfsuid", "setfsgid", "getsid", "ustat", "getpriority", "setpriority", "sched_get_priority_max", "sched_get_priority_min", "_sysctl", "adjtimex", "setrlimit", "settimeofday", "sethostname", "setdomainname", "get_kernel_syms", "gettid", "time", "semtimedop", "clock_settime", "clock_gettime", "clock_getres", "clock_nanosleep", "prlimit64", "clock_adjtime", "getcpu", "shmget", "shmat", "shmctl", "shmdt"]
info_list_arm = ['setuid32', 'setgid32', 'setfsuid32', 'setfsgid32', 'setresuid32', 'getresuid32', 'setresgid32', 'getresgid32', 'setregid32', 'getgroups32', 'setgroups32', 'getuid32', 'getgid32', 'geteuid32', 'getegid32', 'setreuid32', "ugetrlimit", "ipc", "olduname", "profil", "oldolduname", "ulimit", "ftime", "stty", "stime", "nfsservctl", "madvise", "reboot", "kexec_load", "kexec_file_load", "add_key", "request_key", "keyctl", "perf_event_open", "getpid", "uname", "semget", "semop", "semctl", "msgget", "msgctl", "gettimeofday", "getrlimit", "getrusage", "sysinfo", "times", "getuid", "syslog", "getgid", "setuid", "setgid", "geteuid", "getegid", "setpgid", "getppid", "getpgrp", "setsid", "setreuid", "setregid", "getgroups", "setgroups", "setresuid", "getresuid", "setresgid", "getresgid", "getpgid", "setfsuid", "setfsgid", "getsid", "ustat", "getpriority", "setpriority", "sched_get_priority_max", "sched_get_priority_min", "_sysctl", "adjtimex", "setrlimit", "settimeofday", "sethostname", "setdomainname", "get_kernel_syms", "gettid", "time", "semtimedop", "clock_settime", "clock_gettime", "clock_getres", "clock_nanosleep", "prlimit64", "clock_adjtime", "getcpu", "shmget", "shmat", "shmctl", "shmdt"]
info_list = list(set(info_list+info_list_arm))

commu_list = ["mq_open", "mq_unlink", "mq_timedsend", "mq_timedreceive", "mq_notify", "mq_getsetattr", "pipe", "socket", "connect", "accept", "sendto", "recvfrom", "sendmsg", "recvmsg", "shutdown", "bind", "listen", "getsockname", "getpeername", "socketpair", "setsockopt", "getsockopt", "msgsnd", "msgrcv", "accept4", "pipe2", "recvmmsg", "sendmmsg"]
commu_list_arm = ["mq_open", "mq_unlink", "mq_timedsend", "mq_timedreceive", "mq_notify", "mq_getsetattr", 'send', 'recv', "socketcall", "pipe", "socket", "connect", "accept", "sendto", "recvfrom", "sendmsg", "recvmsg", "shutdown", "bind", "listen", "getsockname", "getpeername", "socketpair", "setsockopt", "getsockopt", "msgsnd", "msgrcv", "accept4", "pipe2", "recvmmsg", "sendmmsg"]
commu_list = list(set(commu_list+commu_list_arm))

sys_val_dict = {"Porcess": p_list, "File": f_list, "Device": d_list, "Infomation": info_list, "Communication": commu_list}

sys_dict = {'set_thread_area': 1, 'ugetrlimit': 1, 'readlink': 1, 'getcwd': 1, 'setsid': 1, 'uname': 2, 'fcntl': 3, 'fchown': 3, 'fchmod': 3, 'rename': 3, 'waitpid': 6, 'getuid': 6, 'getgid': 6, 'getppid': 6, 'getegid': 6, 'wait4': 6, 'rt_sigreturn': 6, 'prlimit64': 6, 'umask': 6, 'set_tid_address': 7, 'futex': 8, 'getpid': 9, 'set_robust_list': 11, 'arch_prctl': 12, 'geteuid': 12, 'statfs': 12, 'execve': 13, 'exit_group': 13, 'clone': 16, 'write': 28, 'connect': 39, '_newselect': 39, 'getsockopt': 39, 'send': 39, 'brk': 40, 'socket': 40, 'stat': 48, 'access': 66, 'fstat': 78, 'ioctl': 91, 'mprotect': 100, 'mmap': 138, 'openat': 143, 'time': 189, 'fstat64': 299, 'mmap2': 303, 'munmap': 312, 'nanosleep': 336, 'rt_sigaction': 385, 'rt_sigprocmask': 624, 'lseek': 1500, 'open': 2041, 'read': 2101, 'close': 2463}


###### Pre-process Syscall Table ######
process_creation = ['clone', 'fork', 'vfork']
open_file = ['open', 'openat']
write_file = ['write', 'rename']
info = ['uname', 'sysinfo']
send = ['send', 'sendto', 'sendmsg']
recv = ['recv', 'recvfrom', 'recvmsg']
remove = ['unlink', 'unlinkat', 'rmdir']
link = ['symlink', 'symlinkat', 'link', 'linkat']

pre_sys = process_creation + open_file + write_file + info + send + recv + remove + link
pre_preocess_sys =  ['execve', 'open', 'openat', 'read', 'close', 'write', 'rename', 'socket', 'connect', 'send', 'sendto', 'sendmsg', 'recv', 'recvfrom', 'recvmsg', 'ioctl', 'unlink', 'unlinkat', 'rmdir', 'bind', 'mkdir', 'symlink', 'symlinkat', 'link', 'linkat', 'kill', 'ptrace']

for i in pre_sys:
	if(i not in pre_preocess_sys):
		# print("pre_sys:", i)
		pre_preocess_sys.append(i)

print(len(pre_preocess_sys), pre_preocess_sys)

# get the target syscall list #
path = "./CCOE/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin"

# # print("Family:", path.split("/")[2], "Hash:", path.split("/")[3])
# sys_dict = get_sys_dict(path)
# sys_list = list(sys_dict.keys())
# print(len(sys_list), sys_list)

# unhandle_sys = []
# for sys in sys_list:
# 	if(sys in pre_preocess_sys or sys in p_sig or sys == "exit"):
# 		print("Already handle:", sys)
# 		continue
# 	else:
# 		unhandle_sys.append(sys)
# print(len(unhandle_sys), unhandle_sys)

# non_handle_sys = ["brk", "set_thread_area", ]


32 ['execve', 'open', 'openat', 'read', 'close', 'write', 'rename', 'socket', 'connect', 'send', 'sendto', 'sendmsg', 'recv', 'recvfrom', 'recvmsg', 'ioctl', 'unlink', 'unlinkat', 'rmdir', 'bind', 'mkdir', 'symlink', 'symlinkat', 'link', 'linkat', 'kill', 'ptrace', 'clone', 'fork', 'vfork', 'uname', 'sysinfo']


In [14]:
def parse_line(log):
    signal = re.search('--- (SIG.*?) {(.*)}', log)
    term = re.search('[0-9]* \+\+\+ (.*) \+\+\+', log)
    
    info = {}
    
    if signal:
        info['type'] = 'sig'
        info['name'] = signal.group(1)
        info['args'] = signal.group(2)
    elif term:
        info['type'] = 'term'
        info['status'] = term.group(1)
    else: 
        info['type'] = 'sys'
        g = re.search(' (.*?)\((.*)\).*= (.*)', log)
        
        # uncompleted function caused by terminating execution
        if not g:
            return None
        
        info['name'] = g.group(1)
        info['args'] = g.group(2)
        info['return'] = g.group(3)
        info['success'] = True
        info['return_msg'] = None
        
        
        # return status: True for success, False for error (startswith E or ?)
        ret = info['return'].split(' ')
        if len(ret) > 1:
            info['return'] = ret[0]
            info['return_msg'] = ' '.join(ret[1:])
            
            # inspect first char of return description ()
            if ret[1].startswith('E'):
                info['success'] = False
                info['return_msg'] = ' '.join(ret[2:])
                
        if info['return'] == '?':
            info['success'] = False
    
    time = re.search(RE_TIME, log).group(1)
    info['time'] = time
    
    return info

def get_sys_dict(path):
    sys_dict = {}
    traces = sorted(glob.glob(path + '/strace*'))
    # print(traces)

    total_line = 0
    for idx, trace in enumerate(traces):
        
        with open(trace, 'r') as f:
            for line in f:
                # print("Open")
                line = line.strip()
                call = parse_line(line)
                
                if not call:
                    continue

                if call['type'] == 'sys':
                    total_line += 1
                    
                    sys_name = call['name']
                    # print(sys_name)
                    if(sys_name in sys_dict):
                        sys_dict[sys_name] += 1
                    else:
                        sys_dict[sys_name] = 1
    return sys_dict
                    

def get_sys_arrange(sys_dict):

    sys_arrange = {"Porcess": 0, "File": 0, "Device": 0, "Infomation": 0, "Communication": 0}
    for sys_name in sys_dict:

        for sys_type in sys_val_dict:
            sys_list = sys_val_dict[sys_type]

            if(sys_name in sys_list):
                sys_arrange[sys_type] += sys_dict[sys_name]
                break

            if(sys_type == "Communication"):
                print(sys_name)
                print("Error!!!")

    return sys_arrange

path = "./CCOE/Tsunami/7d3855bb09f2f6111d6c71e06e1e6b06dd47b1dade49af0235b220966c2f5be3.bin"
sys_dict = get_sys_dict(path)
sys_list = list(sys_dict.keys())
print(len(sys_list), sys_list)

31 ['execve', 'ioctl', 'rt_sigaction', 'vfork', 'wait4', 'exit', 'brk', 'access', 'openat', 'fstat', 'mmap', 'close', 'read', 'mprotect', 'arch_prctl', 'munmap', 'getuid', 'getgid', 'getpid', 'geteuid', 'getppid', 'stat', 'getegid', 'fcntl', 'dup2', 'clone', 'rt_sigreturn', 'exit_group', 'chdir', 'getdents', 'write']


In [15]:
# Step3 handled sys
original_sys = ['execve', 'open', 'openat', 'read', 'close', 'write', 'rename', 'socket', 'connect', 'send', 'sendto', 'sendmsg', 'recv', 'recvfrom', 'recvmsg', 'ioctl', 'unlink', 'unlinkat', 'rmdir', 'bind', 'mkdir', 'symlink', 'symlinkat', 'link', 'linkat', 'kill', 'ptrace', 'clone', 'fork', 'vfork', 'uname', 'sysinfo']
dofloo_sys = ['brk', 'set_thread_area', 'set_tid_address', 'set_robust_list', 'futex', 'ugetrlimit', 'readlink', 'getcwd',
 'waitpid', 'exit_group', 'access', 'fstat', 'mmap', 'mprotect', 'arch_prctl', 'munmap', 'getuid', 'getgid', 'getpid',
  'geteuid', 'getppid', 'stat', 'getegid', 'wait4', 'prlimit64', 'statfs', 'umask', 'fcntl', 'fchown', 'fchmod', 'setsid',
   'mmap2', '_newselect', 'getsockopt', 'fstat64', 'nanosleep', 'lseek', 'time']

xorddos_sys = ['execve', 'uname', 'brk', 'set_thread_area', 'set_tid_address', 'set_robust_list', 'futex', 'rt_sigaction', 'rt_sigprocmask', 'ugetrlimit', 'readlink', 'clone', 'exit_group', 'setsid', 'open', 'fstat64', 'dup2', 'close', 'stat64', 'gettimeofday', 'lseek', 'read', 'write', 'waitpid', 'nanosleep', 'unlink', 'shmget', 'shmat', 'shmdt', 'symlink', 'mmap2', 'mprotect', 'access', 'openat', 'fstat', 'mmap', 'arch_prctl', 'munmap', 'prlimit64', 'getuid', 'geteuid', 'getgid', 'getegid', 'stat', 'ioctl', 'fcntl', 'getdents', 'lstat', 'pipe', 'wait4', 'getpid', 'getppid', 'rt_sigreturn', 'statfs', 'umask', 'fchown', 'fchmod', 'rename', 'socket', 'sendto', '_newselect', 'fcntl64', 'connect', 'getsockopt', 'mremap', 'time', 'newfstatat', 'setsockopt', 'getsockname', 'sendmsg', 'recvmsg', 'ppoll']
added_xorddos_sys = ['dup2', 'stat64', 'gettimeofday', 'shmget', 'shmat', 'shmdt', 'getdents', 'lstat', 'pipe', 'fcntl64', 'mremap', 'newfstatat', 'setsockopt', 'getsockname', 'ppoll'] 

tsunami_sys = ['execve', 'ioctl', 'rt_sigaction', 'vfork', 'wait4', 'exit', 'brk', 'access', 'openat', 'fstat', 'mmap', 'close', 'read', 'mprotect', 'arch_prctl', 'munmap', 'getuid', 'getgid', 'getpid', 'geteuid', 'getppid', 'stat', 'getegid', 'fcntl', 'dup2', 'clone', 'rt_sigreturn', 'exit_group', 'chdir', 'getdents', 'write']
added_tsunami_sys = ['chdir']

temp_total_sys = original_sys + dofloo_sys + added_xorddos_sys

p_sig = ["rt_sigpending", "rt_sigtimedwait", "rt_sigqueueinfo", "rt_sigsuspend", "sigaltstack", "rt_sigaction", "rt_sigprocmask", "rt_sigreturn", "personality", "rt_tgsigqueueinfo", ]
p_exit = ["exit"]



print("temp total:", len(temp_total_sys), temp_total_sys, "\n")

non_handle_sys = []
needed_handle_sys = []
for sys in tsunami_sys:
    if sys not in temp_total_sys:
        non_handle_sys.append(sys)
        if sys not in p_sig and sys not in p_exit:
            needed_handle_sys.append(sys)
            
print("non_handle_sys:", len(non_handle_sys), non_handle_sys, "\n")
print("needed_handle_sys:", len(needed_handle_sys), needed_handle_sys, "\n")

temp total: 85 ['execve', 'open', 'openat', 'read', 'close', 'write', 'rename', 'socket', 'connect', 'send', 'sendto', 'sendmsg', 'recv', 'recvfrom', 'recvmsg', 'ioctl', 'unlink', 'unlinkat', 'rmdir', 'bind', 'mkdir', 'symlink', 'symlinkat', 'link', 'linkat', 'kill', 'ptrace', 'clone', 'fork', 'vfork', 'uname', 'sysinfo', 'brk', 'set_thread_area', 'set_tid_address', 'set_robust_list', 'futex', 'ugetrlimit', 'readlink', 'getcwd', 'waitpid', 'exit_group', 'access', 'fstat', 'mmap', 'mprotect', 'arch_prctl', 'munmap', 'getuid', 'getgid', 'getpid', 'geteuid', 'getppid', 'stat', 'getegid', 'wait4', 'prlimit64', 'statfs', 'umask', 'fcntl', 'fchown', 'fchmod', 'setsid', 'mmap2', '_newselect', 'getsockopt', 'fstat64', 'nanosleep', 'lseek', 'time', 'dup2', 'stat64', 'gettimeofday', 'shmget', 'shmat', 'shmdt', 'getdents', 'lstat', 'pipe', 'fcntl64', 'mremap', 'newfstatat', 'setsockopt', 'getsockname', 'ppoll'] 

non_handle_sys: 4 ['rt_sigaction', 'exit', 'rt_sigreturn', 'chdir'] 

needed_handle_

In [ ]:
# Statistic for reduction
import os
import re

mal_list = []
path = "./CCOE"
dirs =  os.listdir(path)
total_count = 0

for dir in dirs:
    print("Malware:", dir)
    fail_count = 0
    succ_count = 0
    malware_hash_dirs = os.listdir(path + "/" + dir)
    # print(malware_hash_dirs)
    for hash in malware_hash_dirs:
        malware_path = path + "/" + dir + "/" + hash
        files = os.listdir(malware_path)
        for file in files:
            isMatch = False
            match = re.search("strace.log.*", file)
            if(match != None):
                mal_list.append(malware_path)
                isMatch = True
                succ_count += 1
                # print(match)
                break
        if(not isMatch):
            fail_count += 1

        total_count += 1
    print("Success count:", succ_count)
    print("Fail count:", fail_count)
    print("-"*10)

print("Total smaple:", total_count)
print("Num of success trace:", len(mal_list))
print("Num of fail trace:", total_count - len(mal_list)) 
# print(mal_list)

# XORDDOS x4

In [ ]:

for mal in mal_list:
    mal_fam = mal.split("/")[2]
    mal_hash = mal.split("/")[3]
    print("---", mal_fam, mal_hash, "---")
    graph = AttackGraph(mal)
    graph.create()
    print("Total trace lines:", graph.traces_lines)
    print("Trigger Lines:", graph.trigger_lines)
    print("Trigger percentage:", round(graph.trigger_lines/graph.traces_lines, 4) * 100, "%")
    mapper = TTPLabeler(graph)
    mapper.fit()
    g = graph.draw(True)

In [ ]:
# Step 3 : 研究剩下的 System call，並繪製 ASG (續)
path = './CCOE/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin'

graph = AttackGraph(path)
graph.create()

# Generation of AS Graph & TT Graph

In [15]:
# path = './root_real_network/Backdoor/Dofloo/fb3cf2464f43906cb96523752d74e5ad8e4eccf8ab74890e51fe24266921ef1c'
path = './CCOE/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin'

In [16]:
# Create Attack Scenario Graph
graph = AttackGraph(path)
graph.create()

# Create TT Graph based on AS Graph
mapper = TTPLabeler(graph)
mapper.fit()
g = graph.draw(False)

# Create Visualization Instance
# g = graph.draw(True)

In [19]:
g = graph.draw(False)
mal_name = path.split("/")[2]
# print(mal_name)
# g.render("./output-img/" + mal_name, format="svg")
g.render("./output-img/" + mal_name, format="png")

Dofloo


In [17]:
# Displat TT Graph
g

# Tactics Time Sequence

In [17]:
for data in graph.node_seq:
    idx = data[0]
    from_node = data[1]
    to_node = data[2]
    edge = data[3]
    print(idx, edge.technique, '-', edge.tactic)

1 System Information Discovery - Discovery
2 Command-Line Interface - Execution
3 Command-Line Interface - Execution
4  - 
5 Local Job Scheduling - Persistence
6 Command-Line Interface - Execution
7 Command-Line Interface - Execution
8 Command-Line Interface - Execution
9 Command-Line Interface - Execution
10  - 
11 Local Job Scheduling - Persistence
12 Command-Line Interface - Execution
13 Command-Line Interface - Execution
14  - 
15 Local Job Scheduling - Persistence
16 Command-Line Interface - Execution
17 Command-Line Interface - Execution
18 Local Job Scheduling - Persistence
19 Command-Line Interface - Execution
20 Command-Line Interface - Execution
21 Local Job Scheduling - Persistence
22  - 
23  - 
24  - 
25 System Network Connection Discovery - Discovery
26 System Information Discovery - Discovery
27  - 
28 System Network Configuration Discovery - Discovery
29 System Information Discovery - Discovery
30 System Information Discovery - Discovery
31 System Information Discovery -

# Plot Graphs for Malware Family
批量產生各惡意家族下每隻惡意程式的 TT Graph，以 PDF 格式存

In [19]:
mode = 'root_real_network'
input_folder = f'./{mode}'
output_folder = f'./attack_graphs/{mode}'
malwares = glob.glob(input_folder+'/*/*/*')

In [20]:
# These two malwares generate too many traces that cuased system hang
special_cases = [
    '3848d8910c35d65335950f727434f741167f8072be0a8384caf53a30e507c46e',
    '84a1af5e8c87bf71172a58bca516e8e8929cabea2983f3941c161a97ec2e4d06'
]

In [23]:
big_graphs = []
nodes_status =[]
malware_files_set = set()

# save graph dot file source
def save_dot(dot, path, m_hash):
    os.makedirs(path, exist_ok=True)
    dot.render(f'{path}/{m_hash}')

# save files list that malware used
def save_files(files, path, m_hash):
    with open(f'{path}/{m_hash}.files.txt', 'w') as f:
        for file in files:
            f.write(f'{file}\n')

# generator
def graph_generator(malware_path):
    global malware_files_set, nodes_status, big_graphs
    m_typ, m_fam, m_hash = malware_path.split('/')[-3:]
    
    if m_hash in special_cases:
        return
        
    print('---', m_typ, m_fam, m_hash, '---')
    graph = AttackGraph(malware_path)
    graph.create()
    
    mapper = TTPLabeler(graph)
    mapper.fit()
    
    malware_files_set = malware_files_set.union(set(graph._file_node_map.keys()))
    dot = graph.draw(False, depth = 3)
    
    all_nodes_num = graph.all_nodes_num
    viz_nodes_num = graph.viz_nodes_num
    color_nodes_num = graph.color_nodes_num
    
    nodes_status.append([m_typ, m_fam, m_hash, all_nodes_num, viz_nodes_num, color_nodes_num])
    #return
    
    m_path = f'{output_folder}/{m_typ}/{m_fam}'
    save_dot(dot, m_path, m_hash)
    save_files(list(graph._file_node_map.keys()), m_path, m_hash)
    print('--- Done ---')

In [24]:
# Save graph PDF, dot source and file list for every malware
for m_path in malwares:
    graph_generator(m_path)

--- Exploit Small 6c3eeafd18595daa48bf903430a88750 ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- Exploit Small ebf56b867b18e322c0380c0c8432868b ---
Node before reduction:  4
4
Nodes after reduction:  4
--- Done ---
--- Exploit Small 480a064b302ec6341d8edfa03c184a40 ---
Node before reduction:  4
4
Nodes after reduction:  4
--- Done ---
--- Exploit Small 84d7377101664d072166bcb142b40f63 ---
Node before reduction:  3
3
Nodes after reduction:  3
--- Done ---
--- Exploit Small d923a5549548c5492516c5204d2f6889 ---
Node before reduction:  3
3
Nodes after reduction:  3
--- Done ---
--- Exploit Small 21eade6c9e4f4d3f4bb060fd0e2940dc ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- Exploit Small c7c873f5baad56d2889e0c819d73fae6 ---
Node before reduction:  3
3
Nodes after reduction:  3
--- Done ---
--- Exploit Small b88504fb961a3af44504804a77fb7547e4db3a55ebd49585947b720a3e99aa98 ---
Node before reduction:  1
1
Nodes after reduction:  1
--- D

Node before reduction:  28
Remove Node: 1905
Remove Node: 1906
26
Nodes after reduction:  26
--- Done ---
--- Backdoor Dofloo fa2212b1eb3ef6d349abf9957ca4e8869074f9a5c5cd987e46933f4fdb54c3f6 ---
Node before reduction:  36
Remove Node: 1458
Remove Node: /proc/net/dev
Remove Node: /sys/devices/system/cpu/online
Remove Node: 1457
Remove Node: /proc/stat
31
Nodes after reduction:  31
--- Done ---
--- Backdoor Dofloo 4a5d98f551781b4f3902e838998af3e641d54437a384381426ad9099f3348c97 ---
Node before reduction:  38
Remove Node: 1475
Remove Node: 1476
36
Nodes after reduction:  36
--- Done ---
--- Backdoor Dofloo ce05c038bc7226f538a9fb1f3ff5c9c074158bbcaead40b6bb6e01190dddd402 ---
Node before reduction:  25
25
Nodes after reduction:  25
--- Done ---
--- Backdoor Dofloo 9883752193bf865198a9997bb4067332cea9ffbc59b8f819307ab69bcf74fd7e ---
Node before reduction:  36
Remove Node: /proc/net/dev
Remove Node: 1325
Remove Node: /proc/stat
Remove Node: 1326
Remove Node: /sys/devices/system/cpu/online
31


Node before reduction:  73
Remove Node: 1534
Remove Node: 1507
Remove Node: 1511
Remove Node: 1487
Remove Node: 1531
Remove Node: 1528
Remove Node: 1510
Remove Node: 1513
Remove Node: 1489
Remove Node: 1535
Remove Node: 1533
Remove Node: 1512
Remove Node: 1515
Remove Node: 1491
Remove Node: 1492
Remove Node: 1536
Remove Node: 1514
Remove Node: 1517
Remove Node: 1496
Remove Node: 1516
Remove Node: 1519
Remove Node: 1498
Remove Node: 1493
Remove Node: 1518
Remove Node: 1509
Remove Node: 1501
Remove Node: 1521
Remove Node: 1497
Remove Node: 1520
Remove Node: 1523
Remove Node: 1503
Remove Node: 1499
Remove Node: 1488
Remove Node: 1485
Remove Node: 1522
Remove Node: 1506
Remove Node: 1526
Remove Node: 1502
Remove Node: 1525
Remove Node: 1505
Remove Node: 1529
32
Nodes after reduction:  32
--- Done ---
--- Backdoor Mayday d39e0a61576a797768f8a7dfb6dadb9a51643b9285b79a96eac296f38f567fc5 ---
Node before reduction:  73
Remove Node: 1390
Remove Node: 1380
Remove Node: 1353
Remove Node: 1376
Remo

Node before reduction:  17
17
Nodes after reduction:  17
--- Done ---
--- HEUR_Backdoor Dofloo 37a0d467147fc6190ae5872dad5f6de7b4f994f0e3d8f517e3d3630a77e1a320 ---
Node before reduction:  30
Remove Node: /proc/stat
Remove Node: /sys/devices/system/cpu/online
Remove Node: /proc/net/dev
Remove Node: 1855
26
Nodes after reduction:  26
--- Done ---
--- HEUR_Backdoor Dofloo b0eb520340ba035d05ec6b9b02078a385a1287ca02bd4020bc253af71ea633cc ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Dofloo 5a3590264394a769f16fde75e7fb7b4e6a9e46ef5784ec4738c7991861cc36ff ---
Node before reduction:  17
17
Nodes after reduction:  17
--- Done ---
--- HEUR_Backdoor Dofloo 6e23f57cf507ec97a81a5ad0b9586436d00dec1b0d331741ffe9945f2442a3d3 ---
Node before reduction:  17
17
Nodes after reduction:  17
--- Done ---
--- HEUR_Backdoor Dofloo 1021cedc679d9dba7c189f55f960ae2383c5c7a0607234b7eaad659b8316bd54 ---
Node before reduction:  17
17
Nodes after reduction:  17
--- Done ---


--- Done ---
--- HEUR_Backdoor Mirai 9c708c9b55f3a3ee04cec89a7699b97f9985bcc8311807f756cb2832151bf58b ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Mirai a3e72a7e3374ba23b9801bdce317266416e8370de83c3ab7343711826b73b8b8 ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Mirai b2654ebbe5e15325c595191cc93c500e5dc3bf42018ab7aca6cb62a5501e71ed ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Mirai 38efecbcabd992df226a9e257a39369ac527c8bc39f0f67bd82a99db2e0298df ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Mirai a36f647f80959739c30947ea4f2cc4ae0aa98d252134697108650f8e6ba1650e ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Mirai 97b69ad25a6dff5004b4e1cf5478a5f26dc469dcb4b388a88421add48ee895e6 ---
Node before reduction:  2
2
Nodes after reduction:  2
--- Done ---
--- HEUR_Backdoor Mirai 82c

...

Node before reduction:  75
Remove Node: 1555
Remove Node: 1553
Remove Node: 1556
Remove Node: 1554
Remove Node: 1552
Remove Node: 1550
Remove Node: 1557
68
Nodes after reduction:  66
--- Done ---
--- HEUR_Backdoor Ganiw 3434b4fcf7018e4f72c93ebbbeddb82b531b4b3f22d3822788d0fee646516820 ---
Node before reduction:  74
Remove Node: 1556
Remove Node: 1549
Remove Node: 1555
Remove Node: 1554
Remove Node: 1551
Remove Node: 1553
Remove Node: 1552
67
Nodes after reduction:  65
--- Done ---
--- HEUR_Backdoor Ganiw 3bb30848e0c3e3f1e505fc6e2e0a3aa5 ---
Node before reduction:  74
Remove Node: 1491
Remove Node: 1490
Remove Node: 1493
Remove Node: 1492
Remove Node: 1494
Remove Node: 1488
Remove Node: 1495
67
Nodes after reduction:  65
--- Done ---
--- HEUR_Backdoor Ganiw 56c28ca56f00ff87096575b962ffd8ad66d01f9411b79257a8162c8acfd0a91f ---
Node before reduction:  74
Remove Node: 1554
Remove Node: 1552
Remove Node: 1559
Remove Node: 1557
Remove Node: 1555
Remove Node: 1558
Remove Node: 1556
67
Nodes aft

Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt eba2e642037cd65d993a7ced1ef7f314f64a018312da149c9196cf358bc8b40d ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 556f86d26a3511fadc11083dc822353dd4e6c5838648057b6c989f02af287eaf ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt f1fd8336567bc42898f1e39c6c82569bf3628843eda9c477549ac049ab669986 ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 9e629680b7e58c8a237dc260892614a7c39df6d8acd4201d1bde5a2d0eb8565c ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 85ce551ad1a9037091745eff743f9c7127775f11ce94a61e2c7d3db9da100ca6 ---
Node before reduction:  8
8
Nodes after reduction:  8
--- Done ---
--- HEUR_Backdoor Gafgyt c7eeabc6368dcf7ca943e35d8cfec9200cfb5d901310bf744987a5b97b0bd42f ---
Node before reduction:  8
8
Nodes 

Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt 0c9724b4201346d8b4d8ecd1f9131fdd96290d7f76d29031e9b30bf49f141eb8 ---
Node before reduction:  8
8
Nodes after reduction:  8
--- Done ---
--- HEUR_Backdoor Gafgyt 96a699043f3c90d2742cb1cf1c0746c0bad53e3f412f3f5516920de59fa981b7 ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt df569e8410073a18634ebee856cdb117b5a6a124ce463837115cb39f85d0d89f ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt d25e2506b2a00a946aa02731ef6f8953964f7b9529e73845f4e3324370c49050 ---
Node before reduction:  8
8
Nodes after reduction:  8
--- Done ---
--- HEUR_Backdoor Gafgyt 4aa7e06aa33d568dc39bcd6fdb00c65752bb7807d3ed9bc51c3ab04e81907a1e ---
Node before reduction:  15
15
Nodes after reduction:  15
--- Done ---
--- HEUR_Backdoor Gafgyt 50828217589b9802c8bbd8a24471ef43239ed3fb92027ec1f980dbbc4d4ef601 ---
Node before reduction:  8
8
Nod

Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt af6c57db7fb2c1e23b3424f5234f9b51ec8350b22a8b3974576ad962a65ba51f ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt e87fb2ad725d0da3b0c210f4c0bee512a3141992bc6e3c9662ce8789819d77d0 ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt ea32441715b5833791782f18babe37e9a70bebaf821fd8fb46a31a6144e84d70 ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 4b3b64d4a940d03c1558543d14b54859ad7825778f00abdd6f4e9ba230a66f83 ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt 9be9395c790b87bbe070b9847a436a12a3a064911a0b8ff1acb354c4a0469137 ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt 8c2f2a6aae830d45392656054baad730d925214a6b29a8d90cba788486f973a4 ---
Node before reduction:  1
1
Nodes 

Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt 7b24158a2cb2ed7343c33bfbb833e8720185ac78942cc9bc9566df8fe0287100 ---
Node before reduction:  8
8
Nodes after reduction:  8
--- Done ---
--- HEUR_Backdoor Gafgyt fa36fd49a9a5b50b4347b9f93e27afc1df16aa30e2eabe0edf52c808b9edeca8 ---
Node before reduction:  15
15
Nodes after reduction:  15
--- Done ---
--- HEUR_Backdoor Gafgyt f42a6e654aeab5b4a7133043f7ec9c94325ef348d7b583917622be112ec9ccba ---
Node before reduction:  1
1
Nodes after reduction:  1
--- Done ---
--- HEUR_Backdoor Gafgyt 37df1f457521110dc77dd33a39f204bfdc42e4457fa55c3e686966bbecd29778 ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 8e56f2ff04197eeec91d155162b1901fd4eab2e1b21f5f8826625800b7f7ad0c ---
Node before reduction:  6
6
Nodes after reduction:  6
--- Done ---
--- HEUR_Backdoor Gafgyt 506ff06c3622ce48ad06731eea2d4469dd82784296ffc076e778fc3732df4951 ---
Node before reduction:  1
1
Nod

# Access File Analysis

In [25]:
len(malware_files_set)

1192

In [26]:
endpoints = set()
for file in malware_files_set:
    if re.search(ENDPOINT_REGEX, file):
        endpoints.add(file)

In [27]:
malware_files_set = malware_files_set - endpoints

In [28]:
len(malware_files_set)

873

In [29]:
len(endpoints)

319

In [30]:
root_dict = {}
others = []
for file in malware_files_set:
    if len(file) > 0:
        if file[0] == '/':
            root = '/' + file.split('/')[1]
            if root not in root_dict:
                root_dict[root] = [file]
            else:
                root_dict[root].append(file)
        else:
            others.append(file)
            

In [31]:
root_dict

{'/proc': ['/proc/817/exe',
  '/proc/418/exe',
  '/proc/1969/exe',
  '/proc/1938/exe',
  '/proc/1796/exe',
  '/proc/988/exe',
  '/proc/788/exe',
  '/proc/2055/exe',
  '/proc/797/exe',
  '/proc/1395/exe',
  '/proc/1272/exe',
  '/proc/995/exe',
  '/proc/415/exe',
  '/proc/3148/exe',
  '/proc/1840/exe',
  '/proc/754/exe',
  '/proc/569/exe',
  '/proc/2030/exe',
  '/proc/1847/exe',
  '/proc/2218/exe',
  '/proc/1425/exe',
  '/proc/1298/exe',
  '/proc/402/exe',
  '/proc/1452/exe',
  '/proc/1868/exe',
  '/proc/630/exe',
  '/proc/430/exe',
  '/proc/981/exe',
  '/proc/1379/exe',
  '/proc/1155/exe',
  '/proc/1305/exe',
  '/proc/2077/exe',
  '/proc/2901/exe',
  '/proc/2443/exe',
  '/proc/964/exe',
  '/proc/1537/exe',
  '/proc/1241/exe',
  '/proc/version',
  '/proc/576/exe',
  '/proc/1857/exe',
  '/proc/953/exe',
  '/proc/1843/exe',
  '/proc/1393/exe',
  '/proc/1234/exe',
  '/proc/962/exe',
  '/proc/455/exe',
  '/proc/1091/exe',
  '/proc/818/exe',
  '/proc/454/exe',
  '/proc/882/exe',
  '/proc/761/

In [32]:
others

['vexpress.img',
 'vmlinuz',
 'eth5',
 'TTeatfiles',
 '2',
 'snapshot.raw',
 'ssh',
 'bsd-port',
 'Unknown',
 'eth9',
 'eth7',
 'eth1',
 'test.c',
 'apsh.conf',
 'run.sh',
 '..',
 'uname',
 'eth6',
 'TTswap',
 'qemu_run.sh',
 'eth3',
 'eth0',
 'vmlinuz-3.10.79.0-1-linaro-lsk-vexpress',
 '.',
 'idus.log',
 'nul',
 'initrd.img-3.10.79.0-1-linaro-lsk-vexpress',
 'output',
 'snapshot.img',
 'TTlib',
 'audit',
 'bin',
 'eth2',
 'eth4',
 'sysinfo',
 'eth8',
 'vga.conf',
 'TTdummyfile',
 'initrd.img',
 '.vi324.tmp',
 'TTeatfile',
 'host_share',
 'TTsharefile',
 'stdin']